## Teegid

In [2]:
from collections import defaultdict
import os, os.path
import re
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from estnltk.resolve_layer_dag import make_resolver
from estnltk.taggers import VabamorfTagger
from estnltk.taggers import VabamorfAnalyzer
from estnltk.taggers import VabamorfCorpusTagger, CorpusBasedMorphDisambiguator
from estnltk import Text
from collections import Counter

## Funktsioonid

### Üldstatistika

In [2]:
def yldstatistika(jarjend):
    lauseid = 0
    sonu = 0
    sonu_kirjavahemarkideta = 0
    dokumente = 0
    for tekst in jarjend:
        lauseid += len(tekst.sentences) #mitu lauset
        #kirjavahemärkideta
        for sona in tekst.ud_morph_reduced: #võtame ud kuldstandardist
            if 'X' not in sona.pos: #kui pole muu
                sonu += 1
                if 'Z' not in sona.pos: #kui kirjavahemärgi märgend
                    sonu_kirjavahemarkideta += 1 #liida üks
        dokumente += 1
    print("Lauseid:                 " + str(lauseid))
    print("Sõnu:                    " + str(sonu))
    print("Sõnu kirjavahemärkideta: " + str(sonu_kirjavahemarkideta))
    print("Dokumente:               " + str(dokumente))

### Tundmatud

In [22]:
def tundmatud(jarjend):
    tundmatuid_o = 0
    tundmatuid_kvm_o = 0 #kirjavahemärkideta
    tundmatuid_s = 0
    tundmatuid_kvm_s = 0 #kirjavahemärkideta
    sonu = 0
    sonu_kvm = 0
    ol_pos = []
    slang_pos = []
    posid = []
    hyyd = set()
    #slang_tundmatu = set()
    #ole_tundmatu = set()
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced: #kuldstandard
            i += 1
            if 'X' not in sona.pos: #kui pole muu
                sonu += 1
                posid.append(sona.pos[0])
                if None in tekst.morph_oletamisteta[i].partofspeech: #kui sõna on tundmatu
                    ol_pos.append(sona.pos[0])
                    tundmatuid_o += 1
                    #ole_tundmatu.add(tekst.morph_oletamisteta[i].text)
                    #if None not in tekst.morph_slangisonastikuga[i].partofspeech:
                        #print("\nOletamisega tundmatu, slängisõnastikuga mitte")
                        #print("Ol: " + tekst.morph_oletamisteta[i].text, tekst.morph_oletamisteta[i].partofspeech)
                        #print("Släng: " + tekst.morph_slangisonastikuga[i].text, tekst.morph_slangisonastikuga[i].partofspeech)
                if None in tekst.morph_slangisonastikuga[i].partofspeech: #kui sõna on tundmatu
                    tundmatuid_s += 1
                    slang_pos.append(sona.pos[0])
                    #slang_tundmatu.add(tekst.morph_slangisonastikuga[i].text)
                    #if None not in tekst.morph_oletamisteta[i].partofspeech:
                        #print("\nSlängisõnastikuga tundmatu, oletamisega mitte")
                        #print("Släng: " + tekst.morph_slangisonastikuga[i].text, tekst.morph_slangisonastikuga[i].partofspeech)
                        #print("Ol: " + tekst.morph_oletamisteta[i].text, tekst.morph_oletamisteta[i].partofspeech)
                if 'Z' not in sona.pos: #kui pole kirjavahemärk:
                    sonu_kvm += 1
                    if None in tekst.morph_oletamisteta[i].partofspeech: #kui sõna pole kirjavahemärk aga on tundmatu
                        tundmatuid_kvm_o += 1   
                    if None in tekst.morph_slangisonastikuga[i].partofspeech: #kui sõna pole kirjavahemärk aga on tundmatu
                        tundmatuid_kvm_s += 1
                        #if sona.pos[0] == 'S':
                        #    hyyd.add(sona.text)
                        
                        
    
    ol_pos_c = Counter(ol_pos)
    slang_pos_c = Counter(slang_pos)
    posid_dict = Counter(posid)
    ol_pos_sorted = ol_pos_c.most_common()
    slang_pos_sorted = slang_pos_c.most_common()
    
    print("VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta")
    print("Tundmatuid sõnu: " + str(tundmatuid_o) + " ehk " + str(round(tundmatuid_o/sonu*100, 2)) + "% sõnadest")
    print("Tundmatuid sõnu kirjavahemärkideta: " + str(tundmatuid_kvm_o) + " ehk " + str(round(tundmatuid_kvm_o/sonu_kvm*100, 2)) + "% sõnadest")
    #normitud
    for a in ol_pos_sorted:
        print(a[0] + ": " + str(round(a[1]/posid_dict[a[0]]*100,2)))
    #normimata
    #for a in ol_pos_sorted:
    #    print(a[0] + ": " + str(round(a[1]/tundmatuid_o*100,2))+"%")
    
    print("\nVabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta, aga slängisõnastikuga")
    print("Tundmatuid sõnu: " + str(tundmatuid_s) + " ehk " + str(round(tundmatuid_s/sonu*100, 2)) + "% sõnadest")
    print("Tundmatuid sõnu kirjavahemärkideta: " + str(tundmatuid_kvm_s) + " ehk " + str(round(tundmatuid_kvm_s/sonu_kvm*100, 2)) + "% sõnadest")
    #normitud
    for a in slang_pos_sorted:
        print(a[0] + ": " + str(round(a[1]/posid_dict[a[0]]*100,2)))
    #normimata
    #for a in slang_pos_sorted:
    #    print(a[0] + ": " + str(round(a[1]/tundmatuid_s*100,2))+"%")
    
    #print("\nSõnad, mida slängisõnastikuga ära ei tuntud, aga oletamisteta tunti")
    #for s in slang_tundmatu:
    #    if s not in ole_tundmatu:
    #        print(s)
    #print("\nSõnad, mida oletamisteta ära ei tuntud, aga slängisõnastikuga tunti")
    #for s in ole_tundmatu:
    #    if s not in slang_tundmatu:
    #        print(s)
    #return hyyd

### Pärisnimede lemmad

In [4]:
def parisnimed(jarjend, alakriips = False):
    ks_pn = 0 #kuldstandard pärisnimesid
    #true positive
    olta_tp = 0 #oletamisteta
    pn_tp = 0 #pärisnime oletamiseta
    olga_tp = 0 #oletamistega
    #false negative
    olta_fn = 0
    pn_fn = 0
    olga_fn = 0
    #false positive
    olta_fp = 0
    pn_fp = 0
    olga_fp = 0
    vormid_vale = []
    vormid = []
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            #kui kuldstandardi kohaselt on pärisnimi
            if 'H' in sona.pos:
                vormid.append(sona.form[0])
                lemma = sona.lemma[0]
                if alakriips == False:
                    if '_' in lemma: #kui liitsõna
                        lemma = lemma.replace('_', '') #eemaldame alakriipsu
                ks_pn += 1
                #oletamisteta
                oige_olta = True
                if alakriips == False:
                    rootid = eemalda_jar(tekst.morph_oletamisteta[i].annotations)
                if alakriips == True:
                    rootid = eemalda_jar(tekst.morph_oletamisteta[i].annotations, alakriips = True)
                if lemma not in rootid:
                    olta_fn += 1
                for analyys in rootid:
                    if analyys == lemma:
                        if oige_olta == False:
                            olta_fp += 1
                        if oige_olta == True:
                            olta_tp += 1
                            oige_olta = False
                    if analyys != lemma:
                        olta_fp += 1
                
                #pärisnime oletamiseta        
                oige_pn = True
                if alakriips == False:
                    rootid = eemalda_jar(tekst.morph_parisnimede_oletamiseta[i].annotations)
                if alakriips == True:
                    rootid = eemalda_jar(tekst.morph_parisnimede_oletamiseta[i].annotations, alakriips = True)
                if lemma not in rootid:
                    pn_fn += 1
                for analyys in rootid:
                    if analyys == lemma:
                        if oige_pn == False:
                            pn_fp += 1
                        if oige_pn == True:
                            pn_tp += 1
                            oige_pn = False
                    if analyys != lemma:
                        pn_fp += 1
                        
                #oletamistega        
                oige_olga = True
                if alakriips == False:
                    rootid = eemalda_jar(tekst.morph_oletamistega[i].annotations)
                if alakriips == True:
                    rootid = eemalda_jar(tekst.morph_oletamistega[i].annotations, alakriips = True)
                if lemma not in rootid:
                    olga_fn += 1
                    vormid_vale.append(sona.form[0])
                for analyys in rootid:
                    if analyys == lemma:
                        if oige_olga == False:
                            olga_fp += 1
                        if oige_olga == True:
                            olga_tp += 1
                            oige_olga = False
                    if analyys != lemma:
                        olga_fp += 1
                        
    #oletamisteta arvutused
    saagis_olta = olta_tp/(olta_tp+olta_fn)
    korrektsus_olta = olta_tp/(olta_tp+olta_fp+olta_fn)
    tapsus_olta = olta_tp/(olta_tp+olta_fp)
    f_olta = 2*((saagis_olta*tapsus_olta)/(saagis_olta+tapsus_olta))
    #pärisnime oletamiseta arvutused
    saagis_pn = pn_tp/(pn_tp+pn_fn)
    korrektsus_pn = pn_tp/(pn_tp+pn_fp+pn_fn)
    tapsus_pn = pn_tp/(pn_tp+pn_fp)
    f_pn = 2*((saagis_pn*tapsus_pn)/(saagis_pn+tapsus_pn))
    #oletamistega arvutused
    saagis_olga = olga_tp/(olga_tp+olga_fn)
    korrektsus_olga = olga_tp/(olga_tp+olga_fp+olga_fn)
    tapsus_olga = olga_tp/(olga_tp+olga_fp)
    f_olga = 2*((saagis_olga*tapsus_olga)/(saagis_olga+tapsus_olga))
    
    vormid_dict = Counter(vormid)
    vormid_v_dict = Counter(vormid_vale)
    vormid_v_sorted = vormid_v_dict.most_common()
    
    print("Oletamisteta")
    print("Korrektsus: " + str(korrektsus_olta)+ " ehk " + str(round(korrektsus_olta*100, 2)) + "%")
    print("Saagis: " + str(saagis_olta)+ " ehk " + str(round(saagis_olta*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_olta)+ " ehk " + str(round(tapsus_olta*100, 2)) + "%")
    print("F1-skoor: " + str(f_olta)+ " ehk " + str(round(f_olta*100, 2)) + "%")
    
    print("\nPärisnime oletamiseta")
    print("Korrektsus: " + str(korrektsus_pn)+ " ehk " + str(round(korrektsus_pn*100, 2))+ "%")
    print("Saagis: " + str(saagis_pn)+ " ehk " + str(round(saagis_pn*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_pn)+ " ehk " + str(round(tapsus_pn*100, 2)) + "%")
    print("F1-skoor: " + str(f_pn)+ " ehk " + str(round(f_pn*100, 2)) + "%")
    
    print("\nOletamistega")
    print("Korrektsus: " + str(korrektsus_olga)+ " ehk " + str(round(korrektsus_olga*100, 2)) + "%")
    print("Saagis: " + str(saagis_olga)+ " ehk " + str(round(saagis_olga*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_olga)+ " ehk " + str(round(tapsus_olga*100, 2)) + "%")
    print("F1-skoor: " + str(f_olga)+ " ehk " + str(round(f_olga*100, 2)) + "%")
    
    print("\nNormaliseeritud: ")
    for a in vormid_v_sorted:
        print(a[0] + ": " + str(round(a[1]/vormid_dict[a[0]]*100,2)))
        
    print("\nNormaliseerimata: ")
    for a in vormid_v_sorted:
        print(a[0] + ": " + str(round(a[1]/ks_pn*100,2))+"%")

### Ühesed analüüsid

In [5]:
def analyyse(jarjend):
    sonu = 0
    #oletamistega
    ol_yhesed = 0
    #yhestamisega
    yh_yhesed = 0
    #yhestamisega, aga postanalysisita
    pa_yhesed = 0
    #eelühestamisega
    ey_yhesed = 0
    #järelühestamisega
    jy_yhesed = 0
    #eel- ja järelühestamisega
    ej_yhesed = 0
    #eel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga
    li_yhesed = 0
    #korpusepõhise ühestamisega
    kp_yhesed = 0
    
    for tekst in jarjend:
        i = -1
        #kirjavahemärkideta
        for sona in tekst.ud_morph_reduced: #võtame ud kuldstandardist
            i += 1
            if 'X' not in sona.pos: #kui pole kirjavahemärk
                sonu += 1 #kirjavahemärkideta sõnu
                #oletamistega, aga ilma ühestamiseta
                if len(tekst.morph_oletamistega[i].annotations) == 1:
                    ol_yhesed += 1
                    
                #ühestamisega
                if len(tekst.morph_yhestamisega[i].annotations) == 1:
                    yh_yhesed += 1
                    
                #ühestamisega, aga ilma postanalysis sammuta 
                if len(tekst.morph_postanalysisita[i].annotations) == 1:
                    pa_yhesed += 1
                    
                #eelühestamisega 
                if len(tekst.morph_eelyhestamisega[i].annotations) == 1:
                    ey_yhesed += 1
                    
                #järelühestamisega 
                if len(tekst.morph_jarelyhestamisega[i].annotations) == 1:
                    jy_yhesed += 1
                    
                #eel- ja järelühestamisega 
                if len(tekst.morph_eel_jarel[i].annotations) == 1:
                    ej_yhesed += 1
                    
                #eel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga
                if len(tekst.morph_liit[i].annotations) == 1:
                    li_yhesed += 1
                    
                #korpusepõhine ühestamine
                if len(tekst.morph_korpusepohine[i].annotations) == 1:
                    kp_yhesed += 1
                    
    print("Tavalise ja pärisnimede oletamisega, aga ühestamiseta") 
    print("Üheseid analüüse: " + str(ol_yhesed) + " ehk " + str(round(ol_yhesed/sonu*100, 2)) +"%")
    
    print("\nÜhestamisega")
    print("Üheseid analüüse: "+ str(yh_yhesed) + " ehk " + str(round(yh_yhesed/sonu*100, 2)) +"%")
    
    print("\nÜhestamisega, aga ilma postanalysis sammuta")
    print("Üheseid analüüse: " + str(pa_yhesed) + " ehk " + str(round(pa_yhesed/sonu*100, 2)) +"%")
    
    print("\nEelühestamisega")
    print("Üheseid analüüse: " + str(ey_yhesed) + " ehk " + str(round(ey_yhesed/sonu*100, 2)) +"%")
    
    print("\nJärelühestamisega")
    print("Üheseid analüüse: " + str(jy_yhesed) + " ehk " + str(round(jy_yhesed/sonu*100, 2)) +"%")
    
    print("\nEel- ja järelühestamisega")
    print("Üheseid analüüse: " + str(ej_yhesed) + " ehk " + str(round(ej_yhesed/sonu*100, 2)) +"%")
    
    print("\nEel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga")
    print("Üheseid analüüse: " + str(li_yhesed) + " ehk " + str(round(li_yhesed/sonu*100, 2)) +"%")
    
    print("\nKoprusepõhise ühestamisega")
    print("Üheseid analüüse: " + str(kp_yhesed) + " ehk " + str(round(kp_yhesed/sonu*100, 2)) +"%")

### Eemaldab lemmast eraldajad

In [6]:
def eemalda(analyys, alakriips = False):
    root = analyys.root
    if analyys.partofspeech != 'Z':
        if '_' in root: #kui liitsõna
            if alakriips == False:
                root = root.replace('_', '') #eemaldame alakriipsu
        if '=' in root:
            root = root.replace('=', '')
        if '+' in root:
            root = root.replace('+', '')
    return root

In [7]:
def eemalda_jar(jar, alakriips = False):
    jar_uus = []
    for analyys in jar:
        if analyys.root == None:
            root = 'None'
        if analyys.root != None:
            root = analyys.root
        if analyys.partofspeech != 'Z':
            if '_' in root: #kui liitsõna
                if alakriips == False:
                    root = root.replace('_', '') #eemaldame alakriipsu
            if '=' in root:
                root = root.replace('=', '')
            if '+' in root:
                root = root.replace('+', '')
        jar_uus.append(root)
    return jar_uus

### Korrektne ühene analüüs

In [8]:
def analyys_korrektne(jarjend):
    sonu = 0
    #oletamistega
    ol_yhesed = 0
    ol_lemma = 0
    ol_liik = 0
    ol_vorm = 0
    #yhestamisega
    yh_yhesed = 0
    yh_lemma = 0
    yh_liik = 0
    yh_vorm = 0
    #yhestamisega, aga postanalysisita
    pa_yhesed = 0
    pa_lemma = 0
    pa_liik = 0
    pa_vorm = 0
    #eelühestamisega
    ey_yhesed = 0
    ey_lemma = 0
    ey_liik = 0
    ey_vorm = 0
    #järelühestamisega
    jy_yhesed = 0
    jy_lemma = 0
    jy_liik = 0
    jy_vorm = 0
    #eel- ja järelühestamisega
    ej_yhesed = 0
    ej_lemma = 0
    ej_liik = 0
    ej_vorm = 0
    #eel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga
    li_yhesed = 0
    li_lemma = 0
    li_liik = 0
    li_vorm = 0
    #korpusepõhise ühestamisega
    kp_yhesed = 0
    kp_lemma = 0
    kp_liik = 0
    kp_vorm = 0
    for tekst in jarjend:
        i = -1
        #kirjavahemärkideta
        for sona in tekst.ud_morph_reduced: #võtame ud kuldstandardist
            i += 1
            if 'X' not in sona.pos: #kui pole kirjavahemärk ega muu
                sonu += 1 #kirjavahemärkideta sõnu
                lemma = sona.lemma[0]
                if '_' in lemma: #kui liitsõna
                    lemma = lemma.replace('_', '') #eemaldame alakriipsu
                #oletamistega, aga ilma ühestamiseta
                if len(tekst.morph_oletamistega[i].annotations) == 1:
                    root = eemalda(tekst.morph_oletamistega[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_oletamistega[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_oletamistega[i].form[0]:
                        ol_yhesed += 1
                    if lemma == root:
                        ol_lemma += 1
                    if sona.pos[0] == tekst.morph_oletamistega[i].partofspeech[0]:
                        ol_liik += 1
                    if sona.form[0] == tekst.morph_oletamistega[i].form[0]:
                        ol_vorm += 1
    
                #ühestamisega
                if len(tekst.morph_yhestamisega[i].annotations) == 1:
                    root = eemalda(tekst.morph_yhestamisega[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_yhestamisega[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_yhestamisega[i].form[0]:
                        yh_yhesed += 1
                    if lemma == root:
                        yh_lemma += 1
                    if sona.pos[0] == tekst.morph_yhestamisega[i].partofspeech[0]:
                        yh_liik += 1
                    if sona.form[0] == tekst.morph_yhestamisega[i].form[0]:
                        yh_vorm += 1
                
                    
                #ühestamisega, aga ilma postanalysis sammuta 
                if len(tekst.morph_postanalysisita[i].annotations) == 1:
                    root = eemalda(tekst.morph_postanalysisita[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_postanalysisita[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_postanalysisita[i].form[0]:
                        pa_yhesed += 1
                    if lemma == root:
                        pa_lemma += 1
                    if sona.pos[0] == tekst.morph_postanalysisita[i].partofspeech[0]:
                        pa_liik += 1
                    if sona.form[0] == tekst.morph_postanalysisita[i].form[0]:
                        pa_vorm += 1
                    
                #eelühestamisega 
                if len(tekst.morph_eelyhestamisega[i].annotations) == 1:
                    root = eemalda(tekst.morph_eelyhestamisega[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_eelyhestamisega[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_eelyhestamisega[i].form[0]:
                        ey_yhesed += 1
                    if lemma == root:
                        ey_lemma += 1
                    if sona.pos[0] == tekst.morph_eelyhestamisega[i].partofspeech[0]:
                        ey_liik += 1
                    if sona.form[0] == tekst.morph_eelyhestamisega[i].form[0]:
                        ey_vorm += 1
                    
                #järelühestamisega 
                if len(tekst.morph_jarelyhestamisega[i].annotations) == 1:
                    root = eemalda(tekst.morph_jarelyhestamisega[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_jarelyhestamisega[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_jarelyhestamisega[i].form[0]:
                        jy_yhesed += 1
                    if lemma == root:
                        jy_lemma += 1
                    if sona.pos[0] == tekst.morph_jarelyhestamisega[i].partofspeech[0]:
                        jy_liik += 1
                    if sona.form[0] == tekst.morph_jarelyhestamisega[i].form[0]:
                        jy_vorm += 1
                    
                #eel- ja järelühestamisega 
                if len(tekst.morph_eel_jarel[i].annotations) == 1:
                    root = eemalda(tekst.morph_eel_jarel[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_eel_jarel[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_eel_jarel[i].form[0]:
                        ej_yhesed += 1
                    if lemma == root:
                        ej_lemma += 1
                    if sona.pos[0] == tekst.morph_eel_jarel[i].partofspeech[0]:
                        ej_liik += 1
                    if sona.form[0] == tekst.morph_eel_jarel[i].form[0]:
                        ej_vorm += 1
                    
                #eel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga
                if len(tekst.morph_liit[i].annotations) == 1:
                    root = eemalda(tekst.morph_liit[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_liit[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_liit[i].form[0]:
                        li_yhesed += 1
                    if lemma == root:
                        li_lemma += 1
                    if sona.pos[0] == tekst.morph_liit[i].partofspeech[0]:
                        li_liik += 1
                    if sona.form[0] == tekst.morph_liit[i].form[0]:
                        li_vorm += 1
                    
                #korpusepõhine ühestamine
                if len(tekst.morph_korpusepohine[i].annotations) == 1:
                    root = eemalda(tekst.morph_korpusepohine[i].annotations[0])
                    if lemma == root \
                    and sona.pos[0] == tekst.morph_korpusepohine[i].partofspeech[0] \
                    and sona.form[0] == tekst.morph_korpusepohine[i].form[0]:
                        kp_yhesed += 1
                    if lemma == root:
                        kp_lemma += 1
                    if sona.pos[0] == tekst.morph_korpusepohine[i].partofspeech[0]:
                        kp_liik += 1
                    if sona.form[0] == tekst.morph_korpusepohine[i].form[0]:
                        kp_vorm += 1
                    
    print("Tavalise ja pärisnimede oletamisega, aga ühestamiseta") 
    print("Kõik korrektne: " + str(ol_yhesed) + " ehk " + str(round(ol_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(ol_lemma) + " ehk " + str(round(ol_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(ol_liik) + " ehk " + str(round(ol_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(ol_vorm) + " ehk " + str(round(ol_vorm/sonu*100, 2)) +"%")
    
    print("\nÜhestamisega")
    print("Kõik korrektne: "+ str(yh_yhesed) + " ehk " + str(round(yh_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(yh_lemma) + " ehk " + str(round(yh_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(yh_liik) + " ehk " + str(round(yh_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(yh_vorm) + " ehk " + str(round(yh_vorm/sonu*100, 2)) +"%")
    
    print("\nÜhestamisega, aga ilma postanalysis sammuta")
    print("Kõik korrektne: " + str(pa_yhesed) + " ehk " + str(round(pa_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(pa_lemma) + " ehk " + str(round(pa_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(pa_liik) + " ehk " + str(round(pa_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(pa_vorm) + " ehk " + str(round(pa_vorm/sonu*100, 2)) +"%")
    
    print("\nEelühestamisega")
    print("Kõik korrektne: " + str(ey_yhesed) + " ehk " + str(round(ey_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(ey_lemma) + " ehk " + str(round(ey_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(ey_liik) + " ehk " + str(round(ey_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(ey_vorm) + " ehk " + str(round(ey_vorm/sonu*100, 2)) +"%")
    
    print("\nJärelühestamisega")
    print("Kõik korrektne: " + str(jy_yhesed) + " ehk " + str(round(jy_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(jy_lemma) + " ehk " + str(round(jy_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(jy_liik) + " ehk " + str(round(jy_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(jy_vorm) + " ehk " + str(round(jy_vorm/sonu*100, 2)) +"%")
    
    print("\nEel- ja järelühestamisega")
    print("Kõik korrektne: " + str(ej_yhesed) + " ehk " + str(round(ej_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(ej_lemma) + " ehk " + str(round(ej_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(ej_liik) + " ehk " + str(round(ej_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(ej_vorm) + " ehk " + str(round(ej_vorm/sonu*100, 2)) +"%")
    
    print("\nEel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga")
    print("Kõik korrektne: " + str(li_yhesed) + " ehk " + str(round(li_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(li_lemma) + " ehk " + str(round(li_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(li_liik) + " ehk " + str(round(li_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(li_vorm) + " ehk " + str(round(li_vorm/sonu*100, 2)) +"%")
    
    print("\nKoprusepõhise ühestamisega")
    print("Kõik korrektne: " + str(kp_yhesed) + " ehk " + str(round(kp_yhesed/sonu*100, 2)) +"%")
    print("Lemma korrektne: " + str(kp_lemma) + " ehk " + str(round(kp_lemma/sonu*100, 2)) +"%")
    print("Sõnaliik korrektne: " + str(kp_liik) + " ehk " + str(round(kp_liik/sonu*100, 2)) +"%")
    print("Vorm korrektne: " + str(kp_vorm) + " ehk " + str(round(kp_vorm/sonu*100, 2)) +"%")

### Lemmad

In [9]:
def lemmad(jarjend):
    #true positive
    yh_tp = 0 #ühestamise vaikekonfiguratsioon
    ej_tp = 0 #eel- ja järelühestamisega
    ol_tp = 0 #oletamistega, aga ilma ühestamiseta
    #false negative
    yh_fn = 0
    ej_fn = 0
    ol_fn = 0
    #false positive
    yh_fp = 0
    ej_fp = 0
    ol_fp = 0
    
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            lemma = sona.lemma[0]
            oige_ol = True #esimene õige analüüs
            oige_yh = True
            oige_ej = True
            if 'X' not in sona.pos:
                #ühestamisega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                rootid = eemalda_jar(tekst.morph_yhestamisega[i].annotations, alakriips = True)
                if lemma not in rootid:
                    yh_fn += 1
                for analyys in rootid:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if lemma == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_yh == False:
                            yh_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if oige_yh == True:
                            yh_tp += 1
                            oige_yh = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if lemma != analyys:
                        yh_fp += 1
                            
                #eel- ja järelühestamisega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                rootid = eemalda_jar(tekst.morph_eel_jarel[i].annotations, alakriips = True)
                if lemma not in rootid:
                    ej_fn += 1
                for analyys in rootid:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if lemma == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_ej == False:
                            ej_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if oige_ej == True:
                            ej_tp += 1
                            oige_ej = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if lemma != analyys:
                        ej_fp += 1
                            
                #oletamistega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                rootid = eemalda_jar(tekst.morph_oletamistega[i].annotations, alakriips = True)
                if lemma not in rootid:
                    ol_fn += 1
                for analyys in rootid:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if lemma == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_ol == False:
                            ol_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if oige_ol == True:
                            ol_tp += 1
                            oige_ol = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if lemma != analyys:
                        ol_fp += 1
                    
            
    
    #oletamistega arvutused
    saagis_ol = ol_tp/(ol_tp+ol_fn)
    korrektsus_ol = ol_tp/(ol_tp+ol_fp+ol_fn)
    tapsus_ol = ol_tp/(ol_tp+ol_fp)
    f_ol = 2*((saagis_ol*tapsus_ol)/(saagis_ol+tapsus_ol))
    #ühestamisega arvutused
    saagis_yh = yh_tp/(yh_tp+yh_fn)
    korrektsus_yh = yh_tp/(yh_tp+yh_fp+yh_fn)
    tapsus_yh = yh_tp/(yh_tp+yh_fp)
    f_yh = 2*((saagis_yh*tapsus_yh)/(saagis_yh+tapsus_yh))
    #eel- ja järelühestamisega arvutused
    saagis_ej = ej_tp/(ej_tp+ej_fn)
    korrektsus_ej = ej_tp/(ej_tp+ej_fp+ej_fn)
    tapsus_ej = ej_tp/(ej_tp+ej_fp)
    f_ej = 2*((saagis_ej*tapsus_ej)/(saagis_ej+tapsus_ej))
    
    print("Oletamistega, aga ilma ühestamiseta")
    print("Korrektsus: " + str(korrektsus_ol)+ " ehk " + str(round(korrektsus_ol*100, 2)) + "%")
    print("Saagis: " + str(saagis_ol)+ " ehk " + str(round(saagis_ol*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_ol)+ " ehk " + str(round(tapsus_ol*100, 2)) + "%")
    print("F1-skoor: " + str(f_ol)+ " ehk " + str(round(f_ol*100, 2)) + "%")
    
    print("\nÜhestamisega")
    print("Korrektsus: " + str(korrektsus_yh)+ " ehk " + str(round(korrektsus_yh*100, 2))+ "%")
    print("Saagis: " + str(saagis_yh)+ " ehk " + str(round(saagis_yh*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_yh)+ " ehk " + str(round(tapsus_yh*100, 2)) + "%")
    print("F1-skoor: " + str(f_yh)+ " ehk " + str(round(f_yh*100, 2)) + "%")
    
    print("\nEel- ja järelühestamisega")
    print("Korrektsus: " + str(korrektsus_ej)+ " ehk " + str(round(korrektsus_ej*100, 2)))
    print("Saagis: " + str(saagis_ej)+ " ehk " + str(round(saagis_ej*100, 2)))
    print("Täpsus: " + str(tapsus_ej)+ " ehk " + str(round(tapsus_ej*100, 2)))
    print("F1-skoor: " + str(f_ej)+ " ehk " + str(round(f_ej*100, 2)))
    
    print("\nOletamistega valed: " + str(ol_fn))
    print("Ühestamisega valed: " + str(yh_fn))  

### Sõnaliik

In [10]:
def sonaliik(jarjend):
    #true positive
    yh_tp = 0 #ühestamise vaikekonfiguratsioon
    ej_tp = 0 #eel- ja järelühestamisega
    ol_tp = 0 #oletamistega, aga ilma ühestamiseta
    #false negative
    yh_fn = 0
    ej_fn = 0
    ol_fn = 0
    #false positive
    yh_fp = 0
    ej_fp = 0
    ol_fp = 0
    
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            oige_ol = True #esimene õige analüüs oletamised
            oige_yh = True #esimene õige analüüs ühestamine
            oige_ej = True #esimene õige analüüs eel- ja järelühestamine
            if 'X' not in sona.pos:
                #ühestamisega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli ehk false negative
                if sona.pos[0] not in tekst.morph_yhestamisega[i].partofspeech:
                    yh_fn += 1
                #ühe sõna analüüside kaupa
                for analyys in tekst.morph_yhestamisega[i].partofspeech:
                    #kui kuldtandardi sõnaliik ühtib automaatsega
                    if sona.pos[0] == analyys:
                        #kui analüüs on õige, aga korduv, siis false positive
                        if oige_yh == False:
                            yh_fp += 1
                        #kui analüüs on õige ja esineb esimest korda siis true positive
                        if  oige_yh == True:
                            yh_tp += 1
                            oige_yh = False #määrame oige Falseiks, et see pärast esimest õiget vale oleks
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla ehk false positive
                    if sona.pos[0] != analyys:
                        yh_fp += 1
                            
                #eel- ja järelühestamisega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                if sona.pos[0] not in tekst.morph_eel_jarel[i].partofspeech:
                    ej_fn += 1
                for analyys in tekst.morph_eel_jarel[i].partofspeech:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if sona.pos[0] == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_ej == False:
                            ej_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if  oige_ej == True:
                            ej_tp += 1
                            oige_ej = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if sona.pos[0] != analyys:
                        ej_fp += 1
                            
                #oletamistega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                if sona.pos[0] not in tekst.morph_oletamistega[i].partofspeech:
                    ol_fn += 1
                for analyys in tekst.morph_oletamistega[i].partofspeech:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if sona.pos[0] == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_ol == False:
                            ol_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if  oige_ol == True:
                            ol_tp += 1
                            oige_ol = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if sona.pos[0] != analyys:
                        ol_fp += 1
                    
            
    #oletamistega arvutused
    saagis_ol = ol_tp/(ol_tp+ol_fn)
    korrektsus_ol = ol_tp/(ol_tp+ol_fp+ol_fn)
    tapsus_ol = ol_tp/(ol_tp+ol_fp)
    f_ol = 2*((saagis_ol*tapsus_ol)/(saagis_ol+tapsus_ol))
    #ühestamisega arvutused
    saagis_yh = yh_tp/(yh_tp+yh_fn)
    korrektsus_yh = yh_tp/(yh_tp+yh_fp+yh_fn)
    tapsus_yh = yh_tp/(yh_tp+yh_fp)
    f_yh = 2*((saagis_yh*tapsus_yh)/(saagis_yh+tapsus_yh))
    #eel- ja järelühestamisega arvutused
    saagis_ej = ej_tp/(ej_tp+ej_fn)
    korrektsus_ej = ej_tp/(ej_tp+ej_fp+ej_fn)
    tapsus_ej = ej_tp/(ej_tp+ej_fp)
    f_ej = 2*((saagis_ej*tapsus_ej)/(saagis_ej+tapsus_ej))
    
    print("Oletamistega, aga ilma ühestamiseta")
    print("Korrektsus: " + str(korrektsus_ol)+ " ehk " + str(round(korrektsus_ol*100, 2)) + "%")
    print("Saagis: " + str(saagis_ol)+ " ehk " + str(round(saagis_ol*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_ol)+ " ehk " + str(round(tapsus_ol*100, 2)) + "%")
    print("F1-skoor: " + str(f_ol)+ " ehk " + str(round(f_ol*100, 2)) + "%")
    
    print("\nÜhestamisega")
    print("Korrektsus: " + str(korrektsus_yh)+ " ehk " + str(round(korrektsus_yh*100, 2))+ "%")
    print("Saagis: " + str(saagis_yh)+ " ehk " + str(round(saagis_yh*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_yh)+ " ehk " + str(round(tapsus_yh*100, 2)) + "%")
    print("F1-skoor: " + str(f_yh)+ " ehk " + str(round(f_yh*100, 2)) + "%")
    
    print("\nEel- ja järelühestamisega")
    print("Korrektsus: " + str(korrektsus_ej)+ " ehk " + str(round(korrektsus_ej*100, 2)))
    print("Saagis: " + str(saagis_ej)+ " ehk " + str(round(saagis_ej*100, 2)))
    print("Täpsus: " + str(tapsus_ej)+ " ehk " + str(round(tapsus_ej*100, 2)))
    print("F1-skoor: " + str(f_ej)+ " ehk " + str(round(f_ej*100, 2)))

### Vorm

In [1]:
def vorm(jarjend):
    #true positive
    yh_tp = 0 #ühestamise vaikekonfiguratsioon
    ej_tp = 0 #eel- ja järelühestamisega
    ol_tp = 0 #oletamistega, aga ilma ühestamiseta
    #false negative
    yh_fn = 0
    ej_fn = 0
    ol_fn = 0
    #false positive
    yh_fp = 0
    ej_fp = 0
    ol_fp = 0
    
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            oige_ol = True #esimene õige analüüs
            oige_yh = True
            oige_ej = True
            if 'X' not in sona.pos:
                #ühestamisega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                if sona.form[0] not in tekst.morph_yhestamisega[i].form:
                    yh_fn += 1
                for analyys in tekst.morph_yhestamisega[i].form:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if sona.form[0] == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_yh == False:
                            yh_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if oige_yh == True:
                            yh_tp += 1
                            oige_yh = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if sona.form[0] != analyys:
                        yh_fp += 1
                
                #eel- ja järelühestamisega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                if sona.form[0] not in tekst.morph_eel_jarel[i].form:
                    ej_fn += 1
                for analyys in tekst.morph_eel_jarel[i].form:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if sona.form[0] == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_ej == False:
                            ej_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if oige_ej == True:
                            ej_tp += 1
                            oige_ej = False
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if sona.form[0] != analyys:
                        ej_fp += 1
               
                #oletamistega
                #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                if sona.form[0] not in tekst.morph_oletamistega[i].form:
                    ol_fn += 1
                for analyys in tekst.morph_oletamistega[i].form:
                    #sõnal peab analüüs olema ja see on sinna lisatud
                    if sona.form[0] == analyys:
                        #kui analüüs on õige, aga korduv
                        if oige_ol == False:
                            ol_fp += 1
                        #kui analüüs on õige ja esineb esimest korda
                        if oige_ol == True:
                            ol_tp += 1
                            oige_ol = False
                    #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                    if sona.form[0] != analyys:
                        ol_fp += 1
    
    #oletamistega arvutused
    saagis_ol = ol_tp/(ol_tp+ol_fn)
    korrektsus_ol = ol_tp/(ol_tp+ol_fp+ol_fn)
    tapsus_ol = ol_tp/(ol_tp+ol_fp)
    f_ol = 2*((saagis_ol*tapsus_ol)/(saagis_ol+tapsus_ol))
    #ühestamisega arvutused
    saagis_yh = yh_tp/(yh_tp+yh_fn)
    korrektsus_yh = yh_tp/(yh_tp+yh_fp+yh_fn)
    tapsus_yh = yh_tp/(yh_tp+yh_fp)
    f_yh = 2*((saagis_yh*tapsus_yh)/(saagis_yh+tapsus_yh))
    #eel- ja järelühestamisega arvutused
    saagis_ej = ej_tp/(ej_tp+ej_fn)
    korrektsus_ej = (ej_tp)/(ej_tp+ej_fp+ej_fn)
    tapsus_ej = ej_tp/(ej_tp+ej_fp)
    f_ej = 2*((saagis_ej*tapsus_ej)/(saagis_ej+tapsus_ej))
    
    print("Oletamistega, aga ilma ühestamiseta")
    print("Korrektsus: " + str(korrektsus_ol)+ " ehk " + str(round(korrektsus_ol*100, 2)) + "%")
    print("Saagis: " + str(saagis_ol)+ " ehk " + str(round(saagis_ol*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_ol)+ " ehk " + str(round(tapsus_ol*100, 2)) + "%")
    print("F1-skoor: " + str(f_ol)+ " ehk " + str(round(f_ol*100, 2)) + "%")
    
    print("\nÜhestamisega")
    print("Korrektsus: " + str(korrektsus_yh)+ " ehk " + str(round(korrektsus_yh*100, 2))+ "%")
    print("Saagis: " + str(saagis_yh)+ " ehk " + str(round(saagis_yh*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_yh)+ " ehk " + str(round(tapsus_yh*100, 2)) + "%")
    print("F1-skoor: " + str(f_yh)+ " ehk " + str(round(f_yh*100, 2)) + "%")
    
    print("\nEel- ja järelühestamisega")
    print("Korrektsus: " + str(korrektsus_ej)+ " ehk " + str(round(korrektsus_ej*100, 2)))
    print("Saagis: " + str(saagis_ej)+ " ehk " + str(round(saagis_ej*100, 2)))
    print("Täpsus: " + str(tapsus_ej)+ " ehk " + str(round(tapsus_ej*100, 2)))
    print("F1-skoor: " + str(f_ej)+ " ehk " + str(round(f_ej*100, 2)))

### Analüüsi ja ühestamise koondtulemus (lemma, sõnaliik ja vorm on kõik korrektsed)

In [12]:
def lemma_liik_vorm(jarjend, kirjavahemargid = False):
    #true positive
    yh_tp = 0 #ühestamise vaikekonfiguratsioon
    ej_tp = 0 #eel- ja järelühestamisega
    ol_tp = 0 #oletamistega, aga ilma ühestamiseta
    #false negative
    yh_fn = 0
    ej_fn = 0
    ol_fn = 0
    #false positive
    yh_fp = 0
    ej_fp = 0
    ol_fp = 0
    
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            lemma = sona.lemma[0]
            if kirjavahemargid == False: #kui kasutaja ei taha kirjavahemärkide analüüsi sisse arvestada
                #kui pole kuldstandardi järgi kirjavahemärk ega muu kategooria, mis välja jäi
                if 'Z' not in sona.pos and 'X' not in sona.pos:
                    #ühestamisega
                    rootid = eemalda_jar(tekst.morph_yhestamisega[i].annotations, alakriips = True)
                    j = -1
                    #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                    if lemma not in rootid \
                    or sona.pos[0] not in tekst.morph_yhestamisega[i].partofspeech \
                    or sona.form[0] not in tekst.morph_yhestamisega[i].form:
                        yh_fn += 1
                    for analyys in tekst.morph_yhestamisega[i].annotations:
                        j += 1
                        #sõnal peab analüüs olema ja see on sinna lisatud
                        if sona.form[0] == analyys.form \
                        and sona.pos[0] == analyys.partofspeech and lemma == rootid[j]:
                            yh_tp += 1
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                        if sona.form[0] != analyys.form \
                        or sona.pos[0] != analyys.partofspeech or lemma != rootid[j]:
                            yh_fp += 1
                    #eel- ja järelühestamisega
                    rootid = eemalda_jar(tekst.morph_eel_jarel[i].annotations, alakriips = True)
                    j = -1
                    #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                    if lemma not in rootid \
                    or sona.pos[0] not in tekst.morph_eel_jarel[i].partofspeech \
                    or sona.form[0] not in tekst.morph_eel_jarel[i].form:
                        ej_fn += 1
                    for analyys in tekst.morph_eel_jarel[i].annotations:
                        j += 1
                        #sõnal peab analüüs olema ja see on sinna lisatud
                        if sona.form[0] == analyys.form \
                        and sona.pos[0] == analyys.partofspeech and lemma == rootid[j]:
                            ej_tp += 1
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                        if sona.form[0] != analyys.form \
                        or sona.pos[0] != analyys.partofspeech or lemma != rootid[j]:
                            ej_fp += 1
                    #oletamistega
                    rootid = eemalda_jar(tekst.morph_oletamistega[i].annotations, alakriips = True)
                    j = -1
                    #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                    if lemma not in rootid \
                    or sona.pos[0] not in tekst.morph_oletamistega[i].partofspeech \
                    or sona.form[0] not in tekst.morph_oletamistega[i].form:
                        ol_fn += 1
                    for analyys in tekst.morph_oletamistega[i].annotations:
                        j += 1
                        #sõnal peab analüüs olema ja see on sinna lisatud
                        if sona.form[0] == analyys.form \
                        and sona.pos[0] == analyys.partofspeech and lemma == rootid[j]:
                            ol_tp += 1
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                        if sona.form[0] != analyys.form\
                        or sona.pos[0] != analyys.partofspeech or lemma != rootid[j]:
                            ol_fp += 1
                    
            if kirjavahemargid == True: #kui kasutaja tahab kirjavahemärkide analüüsi sisse arvestada
                if 'X' not in sona.pos: #jätame muu märgendi välja
                    #ühestamisega
                    rootid = eemalda_jar(tekst.morph_yhestamisega[i].annotations, alakriips = True)
                    j = -1
                    #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                    if lemma not in rootid \
                    or sona.pos[0] not in tekst.morph_yhestamisega[i].partofspeech \
                    or sona.form[0] not in tekst.morph_yhestamisega[i].form:
                        yh_fn += 1
                    for analyys in tekst.morph_yhestamisega[i].annotations:
                        j += 1
                        #sõnal peab analüüs olema ja see on sinna lisatud
                        if sona.form[0] == analyys.form \
                        and sona.pos[0] == analyys.partofspeech and lemma == rootid[j]:
                            yh_tp += 1
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                        if sona.form[0] != analyys.form \
                        or sona.pos[0] != analyys.partofspeech or lemma != rootid[j]:
                            yh_fp += 1
                    #eel- ja järelühestamisega
                    rootid = eemalda_jar(tekst.morph_eel_jarel[i].annotations, alakriips = True)
                    j = -1
                    #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                    if lemma not in rootid \
                    or sona.pos[0] not in tekst.morph_eel_jarel[i].partofspeech \
                    or sona.form[0] not in tekst.morph_eel_jarel[i].form:
                        ej_fn += 1
                    for analyys in tekst.morph_eel_jarel[i].annotations:
                        j += 1
                        #sõnal peab analüüs olema ja see on sinna lisatud
                        if sona.form[0] == analyys.form \
                        and sona.pos[0] == analyys.partofspeech and lemma == rootid[j]:
                            ej_tp += 1
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                        if sona.form[0] != analyys.form \
                        or sona.pos[0] != analyys.partofspeech or lemma != rootid[j]:
                            ej_fp += 1
                    #oletamistega
                    rootid = eemalda_jar(tekst.morph_oletamistega[i].annotations, alakriips = True)
                    j = -1
                    #sõnale jäeti analüüs määramata, mida tegelikult vaja oli
                    if lemma not in rootid \
                    or sona.pos[0] not in tekst.morph_oletamistega[i].partofspeech \
                    or sona.form[0] not in tekst.morph_oletamistega[i].form:
                        ol_fn += 1
                    for analyys in tekst.morph_oletamistega[i].annotations:
                        j += 1
                        #sõnal peab analüüs olema ja see on sinna lisatud
                        if sona.form[0] == analyys.form \
                        and sona.pos[0] == analyys.partofspeech and lemma == rootid[j]:
                            ol_tp += 1
                        #sõnale määrati analüüs, mida seal tegelikult ei tohiks olla
                        if sona.form[0] != analyys.form\
                        or sona.pos[0] != analyys.partofspeech or lemma != rootid[j]:
                            ol_fp += 1
    
    #oletamistega arvutused
    saagis_ol = ol_tp/(ol_tp+ol_fn)
    korrektsus_ol = ol_tp/(ol_tp+ol_fp+ol_fn)
    tapsus_ol = ol_tp/(ol_tp+ol_fp)
    f_ol = 2*((saagis_ol*tapsus_ol)/(saagis_ol+tapsus_ol))
    #ühestamisega arvutused
    saagis_yh = yh_tp/(yh_tp+yh_fn)
    korrektsus_yh = yh_tp/(yh_tp+yh_fp+yh_fn)
    tapsus_yh = yh_tp/(yh_tp+yh_fp)
    f_yh = 2*((saagis_yh*tapsus_yh)/(saagis_yh+tapsus_yh))
    #eel- ja järelühestamisega arvutused
    saagis_ej = ej_tp/(ej_tp+ej_fn)
    korrektsus_ej = ej_tp/(ej_tp+ej_fp+ej_fn)
    tapsus_ej = ej_tp/(ej_tp+ej_fp)
    f_ej = 2*((saagis_ej*tapsus_ej)/(saagis_ej+tapsus_ej))
    
    print("Oletamistega, aga ilma ühestamiseta")
    print("Korrektsus: " + str(korrektsus_ol)+ " ehk " + str(round(korrektsus_ol*100, 2)) + "%")
    print("Saagis: " + str(saagis_ol)+ " ehk " + str(round(saagis_ol*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_ol)+ " ehk " + str(round(tapsus_ol*100, 2)) + "%")
    print("F1-skoor: " + str(f_ol)+ " ehk " + str(round(f_ol*100, 2)) + "%")
    
    print("\nÜhestamisega")
    print("Korrektsus: " + str(korrektsus_yh)+ " ehk " + str(round(korrektsus_yh*100, 2))+ "%")
    print("Saagis: " + str(saagis_yh)+ " ehk " + str(round(saagis_yh*100, 2)) + "%")
    print("Täpsus: " + str(tapsus_yh)+ " ehk " + str(round(tapsus_yh*100, 2)) + "%")
    print("F1-skoor: " + str(f_yh)+ " ehk " + str(round(f_yh*100, 2)) + "%")
    
    print("\nEel- ja järelühestamisega")
    print("Korrektsus: " + str(korrektsus_ej)+ " ehk " + str(round(korrektsus_ej*100, 2)))
    print("Saagis: " + str(saagis_ej)+ " ehk " + str(round(saagis_ej*100, 2)))
    print("Täpsus: " + str(tapsus_ej)+ " ehk " + str(round(tapsus_ej*100, 2)))
    print("F1-skoor: " + str(f_ej)+ " ehk " + str(round(f_ej*100, 2)))

### Lemma korrektsus sõnaliikide kaupa

In [13]:
def lemma_korrektsus(kuld, pos, root, posid, posid_neg):
    fn = 0
    fp = 0
    tp = 0
    
    #kui kuldstandardi lemma läheb analüüsi automaatse lemmaga kokku
    if kuld != root:
        #kui selles sõnas pole selle sõnaliigi analüüs juba olnud
        if pos not in posid_neg:
            fn += 1
        fp += 1
    if kuld == root:
        #kui selles sõnas on selle sõnaliigi analüüs juba olnud
        if pos in posid:
            fp += 1
        if pos not in posid:
            tp += 1
    #kui kuldstandardi lemma läheb analüüsi automaatse lemmaga kokku
    if kuld != root:
        #kui selles sõnas pole selle sõnaliigi analüüs juba olnud
        if pos not in posid_neg:
            fn += 1
        fp += 1
    jar = [tp, fp, fn]
    return jar
        

def lemmatiseerimine(jarjend, alakriips = False):
    #TP, FP, FN
    A_jar = [0, 0, 0]
    C_jar = [0, 0, 0]
    U_jar = [0, 0, 0]
    D_jar = [0, 0, 0]
    G_jar = [0, 0, 0]
    H_jar = [0, 0, 0]
    I_jar = [0, 0, 0]
    J_jar = [0, 0, 0]
    K_jar = [0, 0, 0]
    N_jar = [0, 0, 0]
    O_jar = [0, 0, 0]
    P_jar = [0, 0, 0]
    S_jar = [0, 0, 0]
    V_jar = [0, 0, 0]
    Y_jar = [0, 0, 0]
    Z_jar = [0, 0, 0]
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            posid = []
            posid_neg = []
            lemma = sona.lemma[0]
            if alakriips == False:
                if '_' in lemma: #kui liitsõna
                    lemma = lemma.replace('_', '') #eemaldame alakriipsu
            #kui pole kuldstandardi järgi muu kategooria, mis välja jäi
            for analyys in tekst.morph_yhestamisega[i].annotations:
                if alakriips == False:
                    root = eemalda(analyys)
                if alakriips == True:
                    root = eemalda(analyys, alakriips = True)
                pos = analyys.partofspeech
                #et oleks võimalik sõna analüüsides mitu korda esineva sõnaliigiga arvestada
                jar = lemma_korrektsus(lemma, pos, root, posid, posid_neg)
                if lemma == root:
                    posid.append(pos) 
                if lemma != root:
                    posid_neg.append(pos)
                if pos == 'A':
                    A_jar[0] += jar[0]
                    A_jar[1] += jar[1]
                    A_jar[2] += jar[2]
                if pos == 'C':
                    C_jar[0] += jar[0]
                    C_jar[1] += jar[1]
                    C_jar[2] += jar[2]
                if pos == 'U':
                    U_jar[0] += jar[0]
                    U_jar[1] += jar[1]
                    U_jar[2] += jar[2]
                if pos == 'D':
                    D_jar[0] += jar[0]
                    D_jar[1] += jar[1]
                    D_jar[2] += jar[2]
                if pos == 'G':
                    G_jar[0] += jar[0]
                    G_jar[1] += jar[1]
                    G_jar[2] += jar[2]
                if pos == 'H':
                    H_jar[0] += jar[0]
                    H_jar[1] += jar[1]
                    H_jar[2] += jar[2]
                if pos == 'I':
                    I_jar[0] += jar[0]
                    I_jar[1] += jar[1]
                    I_jar[2] += jar[2]
                if pos == 'J':
                    J_jar[0] += jar[0]
                    J_jar[1] += jar[1]
                    J_jar[2] += jar[2]
                if pos == 'K':
                    K_jar[0] += jar[0]
                    K_jar[1] += jar[1]
                    K_jar[2] += jar[2]
                if pos == 'N':
                    N_jar[0] += jar[0]
                    N_jar[1] += jar[1]
                    N_jar[2] += jar[2]
                if pos == 'O':
                    O_jar[0] += jar[0]
                    O_jar[1] += jar[1]
                    O_jar[2] += jar[2]
                if pos == 'P':
                    P_jar[0] += jar[0]
                    P_jar[1] += jar[1]
                    P_jar[2] += jar[2]
                if pos == 'S':
                    S_jar[0] += jar[0]
                    S_jar[1] += jar[1]
                    S_jar[2] += jar[2]
                if pos == 'V':
                    V_jar[0] += jar[0]
                    V_jar[1] += jar[1]
                    V_jar[2] += jar[2]
                if pos == 'Y':
                    Y_jar[0] += jar[0]
                    Y_jar[1] += jar[1]
                    Y_jar[2] += jar[2]
                if pos == 'Z':
                    Z_jar[0] += jar[0]
                    Z_jar[1] += jar[1]
                    Z_jar[2] += jar[2]
                
    #Korrektsused
    print("A: " + str(round(A_jar[0]/(A_jar[0]+A_jar[1]+A_jar[2])*100, 2)) + "%")
    print("C: " + str(round(C_jar[0]/(C_jar[0]+C_jar[1]+C_jar[2])*100, 2)) + "%")
    print("U: " + str(round(U_jar[0]/(U_jar[0]+U_jar[1]+U_jar[2])*100, 2)) + "%")
    print("D: " + str(round(D_jar[0]/(D_jar[0]+D_jar[1]+D_jar[2])*100, 2)) + "%")
    print("G: " + str(round(G_jar[0]/(G_jar[0]+G_jar[1]+G_jar[2])*100, 2)) + "%")
    print("H: " + str(round(H_jar[0]/(H_jar[0]+H_jar[1]+H_jar[2])*100, 2)) + "%")
    print("I: " + str(round(I_jar[0]/(I_jar[0]+I_jar[1]+I_jar[2])*100, 2)) + "%")
    print("J: " + str(round(J_jar[0]/(J_jar[0]+J_jar[1]+J_jar[2])*100, 2)) + "%")
    print("K: " + str(round(K_jar[0]/(K_jar[0]+K_jar[1]+K_jar[2])*100, 2)) + "%")
    print("N: " + str(round(N_jar[0]/(N_jar[0]+N_jar[1]+N_jar[2])*100, 2)) + "%")
    print("O: " + str(round(O_jar[0]/(O_jar[0]+O_jar[1]+O_jar[2])*100, 2)) + "%")
    print("P: " + str(round(P_jar[0]/(P_jar[0]+P_jar[1]+P_jar[2])*100, 2)) + "%")
    print("S: " + str(round(S_jar[0]/(S_jar[0]+S_jar[1]+S_jar[2])*100, 2)) + "%")
    print("V: " + str(round(V_jar[0]/(V_jar[0]+V_jar[1]+V_jar[2])*100, 2)) + "%")
    print("Y: " + str(round(Y_jar[0]/(Y_jar[0]+Y_jar[1]+Y_jar[2])*100, 2)) + "%")
    print("Z: " + str(round(Z_jar[0]/(Z_jar[0]+Z_jar[1]+Z_jar[2])*100, 2)) + "%")
    
    kaandsonad_tp = A_jar[0] + C_jar[0] + U_jar[0] + H_jar[0] + G_jar[0] + N_jar[0] + O_jar[0] + P_jar[0] + S_jar[0]
    kaandsonad_fp = A_jar[1] + C_jar[1] + U_jar[1] + H_jar[1] + G_jar[1] + N_jar[1] + O_jar[1] + P_jar[1] + S_jar[1] 
    kaandsonad_fn = A_jar[2] + C_jar[2] + U_jar[2] + H_jar[2] + G_jar[2] + N_jar[2] + O_jar[2] + P_jar[2] + S_jar[2] 
    
    print("Käändsõnad: " + str(round(kaandsonad_tp/(kaandsonad_tp+kaandsonad_fp+kaandsonad_fn)*100, 2)) + "%")
    print("Tegusõnad: " + str(round(V_jar[0]/(V_jar[0]+V_jar[1]+V_jar[2])*100, 2)) + "%")

### Liitsõnade analüüs

In [14]:
def liitsona(jarjend):
    analyyse = 0
    sama = 0
    eri_liitsonapiir = 0
    kriipsuta_sama_lemma = 0
    erinev = 0
    parisnimi = 0
    nimisona = 0
    
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            kuld = sona.lemma[0]
            if 'X' not in sona.pos:
                if '_' in kuld:
                    for analyys in tekst.morph_yhestamisega[i].annotations:
                        analyyse += 1
                        root = eemalda(analyys, alakriips = True)
                        #sama, mis kuldstandard
                        if kuld == root:
                            sama += 1
                        kuld_kriipsuta = kuld.replace('_', '') #eemaldame alakriipsu
                        analyys_kriipsuta = eemalda(analyys)
                        #on automaatselt liitsõnana analüüsitud
                        if kuld != root and '_' in root:
                            #sama lemma, aga alakriipsud eri kohtades
                            if kuld_kriipsuta == analyys_kriipsuta:
                                eri_liitsonapiir += 1
                        #pole automaatselt liitsõnana analüüsitud, aga sama lemma
                        if kuld_kriipsuta == root:
                            kriipsuta_sama_lemma += 1
                            #print("Kuld: " + kuld + ", automaatne: " + root)
                            if sona.pos[0] == 'H':
                                parisnimi += 1
                            if sona.pos[0] == 'S':
                                nimisona += 1
                        if kuld != root and kuld_kriipsuta != analyys_kriipsuta and kuld_kriipsuta != root:
                            erinev += 1
    
    print("Täpselt sama lemma, alakriipsud samas kohas: " + str(round(sama/analyyse*100, 2)) + "%")
    print("Sama lemma, aga alakriipsud eri kohtades: " + str(round(eri_liitsonapiir/analyyse*100, 2)) + "%")
    print("Ilma kriipsu(de)ta, aga sama lemma: " + str(round(kriipsuta_sama_lemma/analyyse*100, 2)) + "%")
    print("Erinev lemma, kriipsudega või mitte : " + str(round(erinev/analyyse*100, 2)) + "%")
    print("Pole liitsõnana analüüsitud, aga sama lemma: " + str(kriipsuta_sama_lemma))
    print("Pärisnimesid: " + str(parisnimi) + ", nimisõnu: " + str(nimisona))

### Tüüpvead

In [15]:
def vigu(jarjend):
    kaandsonad = ["A", "C", "U", "H", "G", "N", "O", "P", "S"]
    #analüüsivead
    an_koik = 0 #kõik analüüsid on valed
    an_lemma = 0 #lemma on vale
    an_liik = 0 #sõnaliik on vale
    an_vorm = 0 #vorm on vale
    #vigade jaotumised sõnaliigiti ja vormiti
    an_posid_koik = []
    an_vormid_koik_k = []
    an_vormid_koik_p = []
    an_posid_lemma = []
    an_posid_pos = []
    an_vormid_kaand = []
    an_vormid_poord = []
    
    #ühestamisvead
    yh_koik = 0
    yh_lemma = 0
    yh_liik = 0
    yh_vorm = 0
    #vigade jaotumised sõnaliigiti ja vormiti
    yh_posid_koik = []
    yh_vormid_koik_k = []
    yh_vormid_koik_p = []
    yh_posid_lemma = []
    yh_posid_pos = []
    yh_vormid_kaand = []
    yh_vormid_poord = []
    
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            if 'X' not in sona.pos:
                lemma = sona.lemma[0]
                pos = sona.pos[0]
                vorm = sona.form[0]
                #oletamistega, aga ilma ühestamiseta
                rootid = eemalda_jar(tekst.morph_oletamistega[i].annotations, alakriips=True)
                if lemma not in rootid \
                and pos not in tekst.morph_oletamistega[i].partofspeech \
                and vorm not in tekst.morph_oletamistega[i].form:
                    an_koik += 1
                    an_posid_koik.append(pos)
                    if pos in kaandsonad:
                        an_vormid_koik_k.append(vorm)
                    if pos == 'V':
                        an_vormid_koik_p.append(vorm)
                if lemma not in rootid:
                    an_lemma += 1
                    an_posid_lemma.append(pos)
                if pos not in tekst.morph_oletamistega[i].partofspeech:
                    an_liik += 1
                    an_posid_pos.append(pos)
                if vorm not in tekst.morph_oletamistega[i].form:
                    an_vorm += 1
                    if pos in kaandsonad:
                        an_vormid_kaand.append(vorm)
                    if pos == 'V':
                        an_vormid_poord.append(vorm)
    
                #ühestamisega
                rootid_yh = eemalda_jar(tekst.morph_yhestamisega[i].annotations, alakriips=True)
                if lemma not in rootid_yh and lemma in rootid \
                and sona.pos[0] not in tekst.morph_yhestamisega[i].partofspeech \
                and sona.pos[0] in tekst.morph_oletamistega[i].partofspeech \
                and sona.form[0] not in tekst.morph_yhestamisega[i].form \
                and sona.form[0] in tekst.morph_oletamistega[i].form:
                    yh_koik += 1
                    yh_posid_koik.append(pos)
                    if pos in kaandsonad:
                        yh_vormid_koik_k.append(vorm)
                    if pos == 'V':
                        yh_vormid_koik_p.append(vorm)
                if lemma not in rootid_yh and lemma in rootid:
                    yh_lemma += 1
                    yh_posid_lemma.append(pos)
                if sona.pos[0] not in tekst.morph_yhestamisega[i].partofspeech and sona.pos[0] in tekst.morph_oletamistega[i].partofspeech:
                    yh_liik += 1
                    yh_posid_pos.append(pos)
                if sona.form[0] not in tekst.morph_yhestamisega[i].form and sona.form[0] in tekst.morph_oletamistega[i].form:
                    yh_vorm += 1
                    if pos in kaandsonad:
                        yh_vormid_kaand.append(vorm)
                    if pos == 'V':
                        yh_vormid_poord.append(vorm)
    
    viga_koik = an_koik + yh_koik
    viga_lemma = an_lemma + yh_lemma
    viga_liik = an_liik + yh_liik
    viga_vorm = an_vorm + yh_vorm
    viga_suur = viga_lemma + viga_liik + viga_vorm
    viga_suur_yh = yh_lemma+yh_liik+yh_vorm
    viga_suur_an = an_lemma+an_liik+an_vorm
    
    #käänd ja pöördsõnade osakaal valedest vormidest
    an_kaande_k = len(an_vormid_koik_k)
    an_poorde_k = len(an_vormid_koik_p)
    an_kaande = len(an_vormid_kaand)
    an_poorde = len(an_vormid_poord)
    
    yh_kaande_k = len(yh_vormid_koik_k)
    yh_poorde_k = len(yh_vormid_koik_p)
    yh_kaande = len(yh_vormid_kaand)
    yh_poorde = len(yh_vormid_poord)
    
    #loendame, mitu korda eri sõnaliike ja vorme esines
    an_posid_koik2 = Counter(an_posid_koik)
    an_vormid_koik_k2 = Counter(an_vormid_koik_k)
    an_vormid_koik_p2 = Counter(an_vormid_koik_p)
    an_posid_lemma2 = Counter(an_posid_lemma)
    an_posid_pos2 = Counter(an_posid_pos)
    an_vormid_kaand2 = Counter(an_vormid_kaand)
    an_vormid_poord2 = Counter(an_vormid_poord)
    
    yh_posid_koik2 = Counter(yh_posid_koik)
    yh_vormid_koik_k2 = Counter(yh_vormid_koik_k)
    yh_vormid_koik_p2 = Counter(yh_vormid_koik_p)
    yh_posid_lemma2 = Counter(yh_posid_lemma)
    yh_posid_pos2 = Counter(yh_posid_pos)
    yh_vormid_kaand2 = Counter(yh_vormid_kaand)
    yh_vormid_poord2 = Counter(yh_vormid_poord)
    
    #leiame 5 kõige sagedasemat
    an_p_k = an_posid_koik2.most_common(5)
    an_v_k_k = an_vormid_koik_k2.most_common(5)
    an_v_k_p = an_vormid_koik_p2.most_common(5)
    an_p_l = an_posid_lemma2.most_common(5)
    an_p_p = an_posid_pos2.most_common(5)
    an_v_k = an_vormid_kaand2.most_common(5)
    an_v_p = an_vormid_poord2.most_common(5)

    yh_p_k = yh_posid_koik2.most_common(5)
    yh_v_k_k = yh_vormid_koik_k2.most_common(5)
    yh_v_k_p = yh_vormid_koik_p2.most_common(5)
    yh_p_l = yh_posid_lemma2.most_common(5)
    yh_p_p = yh_posid_pos2.most_common(5)
    yh_v_k = yh_vormid_kaand2.most_common(5)
    yh_v_p = yh_vormid_poord2.most_common(5)
    
    print("Kõik kategooriad olid valed: " + str(viga_koik))
    print("Analüüsivigu: " + str(an_koik) + " ehk " + str(round(an_koik/viga_koik*100, 2)) + "%")
    for a in an_p_k:
        print(a[0] + ": " + str(round(a[1]/an_koik*100,2))+"%")
    print("\nKäänded: " + str(round(an_kaande_k/an_koik*100, 2))+"%")
    for a in an_v_k_k:
        print(a[0] + ": " + str(round(a[1]/an_koik*100,2))+"%")
    print("\nPöörded: " + str(round(an_poorde_k/an_koik*100, 2))+"%")
    for a in an_v_k_p:
        print(a[0] + ": " + str(round(a[1]/an_koik*100,2))+"%")
    print("\n**********************************************")
    print("\nÜhestamisvigu: " + str(yh_koik) + " ehk " + str(round(yh_koik/viga_koik*100, 2))+ "%")
    for a in yh_p_k:
        print(a[0] + ": " + str(round(a[1]/yh_koik*100,2))+"%")
    print("\nKäänded: " + str(round(yh_kaande_k/yh_koik*100, 2))+"%")
    for a in yh_v_k_k:
        print(a[0] + ": " + str(round(a[1]/yh_koik*100,2))+"%")
    print("\nPöörded: " + str(round(yh_poorde_k/yh_koik*100, 2))+"%")
    for a in yh_v_k_p:
        print(a[0] + ": " + str(round(a[1]/yh_koik*100,2))+"%")
    
    print("\n----------------------------------------------------------------------------------------------")
    print("\nLemma vale: " + str(viga_lemma))
    print("Analüüsivigu: " + str(an_lemma) + " ehk " + str(round(an_lemma/viga_lemma*100, 2))+ "%")
    for a in an_p_l:
        print(a[0] + ": " + str(round(a[1]/an_lemma*100,2))+"%")
    print("\nÜhestamisvigu: " + str(yh_lemma) + " ehk " + str(round(yh_lemma/viga_lemma*100, 2))+ "%")
    for a in yh_p_l:
        print(a[0] + ": " + str(round(a[1]/yh_lemma*100,2))+"%")
    
    print("\n----------------------------------------------------------------------------------------------")
    print("\nSõnaliik vale: " + str(viga_liik))
    print("Analüüsivigu: " + str(an_liik) + " ehk " + str(round(an_liik/viga_liik*100, 2))+ "%")
    for a in an_p_p:
        print(a[0] + ": " + str(round(a[1]/an_liik*100,2))+"%")
    print("\nÜhestamisvigu: " + str(yh_liik) + " ehk " + str(round(yh_liik/viga_liik*100, 2))+ "%")
    for a in yh_p_p:
        print(a[0] + ": " + str(round(a[1]/yh_liik*100,2))+"%")
    
    print("\n----------------------------------------------------------------------------------------------")
    print("\nVorm vale: " + str(viga_vorm))
    print("Analüüsivigu: " + str(an_vorm) + " ehk " + str(round(an_vorm/viga_vorm*100, 2))+ "%")
    print("\nKäänded: " + str(round(an_kaande/an_vorm*100, 2))+"%")
    for a in an_v_k:
        print(a[0] + ": " + str(round(a[1]/an_vorm*100,2))+"%")
    print("\nPöörded: " + str(round(an_poorde/an_vorm*100, 2))+"%")
    for a in an_v_p:
        print(a[0] + ": " + str(round(a[1]/an_vorm*100,2))+"%")
    print("\n**********************************************")
    print("\nÜhestamisvigu: " + str(yh_vorm) + " ehk " + str(round(yh_vorm/viga_vorm*100, 2))+ "%")
    print("\nKäänded: " + str(round(yh_kaande/yh_vorm*100, 2))+"%")
    for a in yh_v_k:
        print(a[0] + ": " + str(round(a[1]/yh_vorm*100,2))+"%")
    print("\nPöörded: " + str(round(yh_poorde/yh_vorm*100, 2))+"%")
    for a in yh_v_p:
        print(a[0] + ": " + str(round(a[1]/yh_vorm*100,2))+"%")
    
    print("\n----------------------------------------------------------------------------------------------")
    print("\nLemma, liik või vorm vale: " + str(viga_suur))
    print("Analüüsivigu: " + str(viga_suur_an) + " ehk " + str(round(viga_suur_an/viga_suur*100, 2))+ "%")
    print("Ühestamisvigu: " + str(viga_suur_yh) + " ehk " + str(round(viga_suur_yh/viga_suur*100, 2))+ "%")

### Sõnaliigi vead

In [16]:
def vigu_liik(jarjend):
    posid = []
    posid_vale = []
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            posid.append(sona.pos[0])
            if sona.pos[0] not in tekst.morph_oletamistega[i].partofspeech:
                posid_vale.append(sona.pos[0])
    
    posid_dict = Counter(posid)
    posid_v_dict = Counter(posid_vale)
    posid_v_sorted = posid_v_dict.most_common()
    
    print("\nNormaliseeritud: ")
    for a in posid_v_sorted:
        print(a[0] + ": " + str(round(a[1]/posid_dict[a[0]]*100,2)))
        
    print("\nNormaliseerimata: ")
    for a in posid_v_sorted:
        print(a[0] + ": " + str(round(a[1]/len(posid)*100,2))+"%")

### Vormi vead

In [17]:
def vigu_vorm(jarjend):
    posid = []
    posid_vale = []
    for tekst in jarjend:
        i = -1
        for sona in tekst.ud_morph_reduced:
            i += 1
            posid.append(sona.form[0])
            if sona.form[0] not in tekst.morph_oletamistega[i].form:
                posid_vale.append(sona.form[0])
    
    posid_dict = Counter(posid)
    posid_v_dict = Counter(posid_vale)
    posid_v_sorted = posid_v_dict.most_common()
    
    print("\nNormaliseeritud: ")
    for a in posid_v_sorted:
        print(a[0] + ": " + str(round(a[1]/posid_dict[a[0]]*100,2)))
        
    print("\nNormaliseerimata: ")
    for a in posid_v_sorted:
        print(a[0] + ": " + str(round(a[1]/len(posid)*100,2))+"%")

### Sõnaliikide osakaalud

In [18]:
def sonaliikide_jaotus(jarjend):
    sonu = 0
    a = 0
    c = 0
    u = 0
    d = 0
    g = 0
    h = 0
    i = 0
    i = 0
    j = 0
    k = 0
    n = 0
    o = 0
    p = 0
    s = 0
    v = 0
    y = 0
    z = 0
    for tekst in jarjend:
        for sona in tekst.ud_morph_reduced:
            pos = sona.pos[0]
            sonu += 1
            if pos == 'A':
                a += 1 
            if pos == 'C':
                c += 1    
            if pos == 'U':
                u += 1
            if pos == 'D':
                d += 1
            if pos == 'G':
                g += 1  
            if pos == 'H':
                h += 1
            if pos == 'I':
                i += 1
            if pos == 'J':
                j += 1
            if pos == 'K':
                k += 1
            if pos == 'N':
                n += 1
            if pos == 'O':
                o += 1
            if pos == 'P':
                p += 1
            if pos == 'S':
                s += 1
            if pos == 'V':
                v += 1
            if pos == 'Y':
                y += 1
            if pos == 'Z':
                z += 1
            if pos == 'X':
                sonu -= 1
    
    print("A: " + str(a) + " ehk " + str(round(a/sonu*100, 2))+ "%")
    print("C: " + str(c) + " ehk " + str(round(c/sonu*100, 2))+ "%")
    print("U: " + str(u) + " ehk " + str(round(u/sonu*100, 2))+ "%")
    print("D: " + str(d) + " ehk " + str(round(d/sonu*100, 2))+ "%")
    print("G: " + str(g) + " ehk " + str(round(g/sonu*100, 2))+ "%")
    print("H: " + str(h) + " ehk " + str(round(h/sonu*100, 2))+ "%")
    print("I: " + str(i) + " ehk " + str(round(i/sonu*100, 2))+ "%")
    print("J: " + str(j) + " ehk " + str(round(j/sonu*100, 2))+ "%")
    print("K: " + str(k) + " ehk " + str(round(k/sonu*100, 2))+ "%")
    print("N: " + str(n) + " ehk " + str(round(n/sonu*100, 2))+ "%")
    print("O: " + str(o) + " ehk " + str(round(o/sonu*100, 2))+ "%")
    print("P: " + str(p) + " ehk " + str(round(p/sonu*100, 2))+ "%")
    print("S: " + str(s) + " ehk " + str(round(s/sonu*100, 2))+ "%")
    print("V: " + str(v) + " ehk " + str(round(v/sonu*100, 2))+ "%")
    print("Y: " + str(y) + " ehk " + str(round(y/sonu*100, 2))+ "%")
    print("Z: " + str(z) + " ehk " + str(round(z/sonu*100, 2))+ "%")
    
    kaandsonad = a + c + u + h + g + n + o + p + s
    print("Käändsõnad: " + str(kaandsonad) + " ehk " + str(round(kaandsonad/sonu*100, 2))+ "%")

## Laadi automaatmärgendustega failid sisse ja pane vastavalt žanrile listidesse

### EDT korpus

In [3]:
# Corpus with gold standard annotations and automaatmärgendus
input_dir = 'Automaatmargendusega_EDT_hastaalfinaal'
assert os.path.isdir( input_dir )
loaded_texts_edt_am = []
for fname in os.listdir( input_dir ):
    #if 'dev' in fname:
        #continue
    #if 'test' in fname:
     #   continue
    #if 'train' in fname:
     #   continue
    if fname.endswith('.json'):
        # Load Text with gold standard annotations
        text = json_to_text(file=os.path.join(input_dir, fname) )
        loaded_texts_edt_am.append( text )
        print(' Loaded ', fname)

 Loaded  et_edt-ud-dev_000_am.json
 Loaded  et_edt-ud-dev_001_am.json
 Loaded  et_edt-ud-dev_002_am.json
 Loaded  et_edt-ud-dev_003_am.json
 Loaded  et_edt-ud-dev_004_am.json
 Loaded  et_edt-ud-dev_006_am.json
 Loaded  et_edt-ud-dev_007_am.json
 Loaded  et_edt-ud-dev_008_am.json
 Loaded  et_edt-ud-test_009_am.json
 Loaded  et_edt-ud-test_010_am.json
 Loaded  et_edt-ud-test_011_am.json
 Loaded  et_edt-ud-test_013_am.json
 Loaded  et_edt-ud-test_014_am.json
 Loaded  et_edt-ud-train_015_am.json
 Loaded  et_edt-ud-train_016_am.json
 Loaded  et_edt-ud-train_017_am.json
 Loaded  et_edt-ud-train_018_am.json
 Loaded  et_edt-ud-train_019_am.json
 Loaded  et_edt-ud-train_020_am.json
 Loaded  et_edt-ud-train_021_am.json
 Loaded  et_edt-ud-train_022_am.json
 Loaded  et_edt-ud-train_023_am.json
 Loaded  et_edt-ud-train_024_am.json
 Loaded  et_edt-ud-train_025_am.json
 Loaded  et_edt-ud-train_027_am.json
 Loaded  et_edt-ud-train_028_am.json
 Loaded  et_edt-ud-train_029_am.json
 Loaded  et_edt-ud-tra

In [4]:
#jaotab korpuse žanripõhiselt eraldi listidesse
aja = [] #ajakirjandus
tea = [] #teadus
ilu = [] #ilukirjandus
for t in loaded_texts_edt_am:
    file_id = t.meta['file_prefix']
    if re.match('^tea', file_id):
        tea.append(t)
    elif re.match('^aja', file_id):
        aja.append(t)
    elif re.match('^ilu', file_id):
        ilu.append(t)

### EWT korpus

In [3]:
# Corpus with gold standard annotations and automaatmärgendus
input_dir = 'Automaatmargendusega_EWT_hastaalfinaal'
assert os.path.isdir( input_dir )
loaded_texts_ewt_am = []
for fname in os.listdir( input_dir ):
    #if 'dev' in fname:
        #continue
    #if 'test' in fname:
     #   continue
    #if 'train' in fname:
     #   continue
    if fname.endswith('.json'):
        # Load Text with gold standard annotations
        text = json_to_text(file=os.path.join(input_dir, fname) )
        loaded_texts_ewt_am.append( text )
        print(' Loaded ', fname)

 Loaded  et_ewt-ud-dev_000_am.json
 Loaded  et_ewt-ud-dev_001_am.json
 Loaded  et_ewt-ud-dev_002_am.json
 Loaded  et_ewt-ud-dev_003_am.json
 Loaded  et_ewt-ud-dev_004_am.json
 Loaded  et_ewt-ud-test_005_am.json
 Loaded  et_ewt-ud-test_006_am.json
 Loaded  et_ewt-ud-test_007_am.json
 Loaded  et_ewt-ud-test_008_am.json
 Loaded  et_ewt-ud-test_009_am.json
 Loaded  et_ewt-ud-test_010_am.json
 Loaded  et_ewt-ud-test_011_am.json
 Loaded  et_ewt-ud-train_012_am.json
 Loaded  et_ewt-ud-train_013_am.json
 Loaded  et_ewt-ud-train_014_am.json
 Loaded  et_ewt-ud-train_015_am.json
 Loaded  et_ewt-ud-train_016_am.json
 Loaded  et_ewt-ud-train_017_am.json
 Loaded  et_ewt-ud-train_018_am.json
 Loaded  et_ewt-ud-train_019_am.json
 Loaded  et_ewt-ud-train_020_am.json
 Loaded  et_ewt-ud-train_021_am.json
 Loaded  et_ewt-ud-train_022_am.json
 Loaded  et_ewt-ud-train_023_am.json
 Loaded  et_ewt-ud-train_024_am.json
 Loaded  et_ewt-ud-train_025_am.json
 Loaded  et_ewt-ud-train_026_am.json
 Loaded  et_ewt-ud

In [6]:
foorum = [] 
ewtb = [] #estonian web treebank (vanad tekstid)
kom = [] #kommentaarium
for t in loaded_texts_ewt_am:
    file_id = t.meta['file_prefix']
    if 'foorum' in file_id:
        foorum.append(t)
    elif 'ewtb' in file_id:
        ewtb.append(t)
    elif 'kom' in file_id:
        kom.append(t)

In [7]:
kogu_korpus = loaded_texts_edt_am + loaded_texts_ewt_am

## Korpusepõhiste konfiguratsioonidega tagimine

In [ ]:
#korpusepõhine ühestamine
#NB! Tekstid tuleb enne tagimist grupeerida
korpusepohine = VabamorfCorpusTagger( output_layer='morph_korpusepohine' )

In [ ]:
#VabamorfTagger koos tekstipõhise eelühestamise- ja järelühestamisega ning liitsõnade ühestamise heuristikuga. 
cb_disambiguator = CorpusBasedMorphDisambiguator( output_layer='morph_liit', 
                                                  disamb_compound_words=True )
 
eel_jarel_liit = VabamorfCorpusTagger(
    output_layer='morph_liit',
    disambiguate=True,
    guess=True, 
    propername=True,
    slang_lex=False,
    use_postanalysis=True,
    predisambiguate=True,
    postdisambiguate=True,
    cb_disambiguator=cb_disambiguator
    )

In [42]:
eel_jarel_liit.tag(aja)
korpusepohine.tag(aja)
eel_jarel_liit.tag(tea)
korpusepohine.tag(tea)
eel_jarel_liit.tag(ilu)
korpusepohine.tag(ilu)

eel_jarel_liit.tag(foorum)
korpusepohine.tag(foorum)
eel_jarel_liit.tag(ewtb)
korpusepohine.tag(ewtb)
eel_jarel_liit.tag(kom)
korpusepohine.tag(kom)

edt_korpusepohistega = aja + ilu + tea
ewt_korpusepohistega = foorum + ewtb + kom

In [30]:
kogu_korpus = edt_korpusepohistega + ewt_korpusepohistega

# Tüüpvead

## Vormi vigade jaotumine

In [90]:
print("EDT")
vigu_vorm(loaded_texts_edt_am)
print("\nEWT")
vigu_vorm(loaded_texts_ewt_am)
print("\nKogu koprus")
vigu_vorm(kogu_korpus)

EDT

Normaliseeritud: 
: 4.9
sg n: 6.01
neg o: 72.41
neg nud: 82.23
sg g: 2.23
sg ad: 6.48
neg ks: 72.96
sg el: 2.76
sg p: 0.68
sg in: 1.12
pl g: 0.61
sg all: 1.06
sg tr: 0.87
adt: 1.6
pl ad: 1.36
pl n: 0.19
pl p: 0.33
neg: 0.65
sg kom: 0.41
sg ter: 3.07
neg ge: 52.17
sg abl: 1.68
b: 0.06
pl el: 0.47
nud: 0.23
sg ill: 0.79
vad: 0.15
s: 0.05
tud: 0.31
neg vat: 50.0
da: 0.07
pl in: 0.33
des: 0.3
sin: 0.38
ti: 0.35
ma: 0.16
takse: 0.21
neg nuks: 100.0
neg gem: 50.0
ks: 0.14
o: 0.49
pl kom: 0.17
mas: 0.74
gu: 2.7
pl all: 0.12
v: 100.0
pl ill: 0.29
sid: 0.04
te: 0.49
sg es: 0.13
me: 0.14
d: 0.26
neg gu: 50.0
taks: 3.33

Normaliseerimata: 
: 1.8%
sg n: 0.81%
neg o: 0.59%
neg nud: 0.23%
sg g: 0.22%
sg ad: 0.13%
neg ks: 0.05%
sg el: 0.03%
sg p: 0.03%
sg in: 0.02%
pl g: 0.02%
sg all: 0.01%
sg tr: 0.01%
adt: 0.01%
pl ad: 0.01%
pl n: 0.01%
pl p: 0.01%
neg: 0.01%
sg kom: 0.0%
sg ter: 0.0%
neg ge: 0.0%
sg abl: 0.0%
b: 0.0%
pl el: 0.0%
nud: 0.0%
sg ill: 0.0%
vad: 0.0%
s: 0.0%
tud: 0.0%
neg vat: 0.0%

### Analüüsivigade näited

In [122]:
valed = []
posid = []
for tekst in kogu_korpus:
    i = -1 
    for sona in tekst.ud_morph_reduced:
        i += 1
        vormid = tekst.morph_oletamistega[i].form
        if sona.form[0] not in vormid and 'sg ad' == sona.form[0]:
            print("\n" + tekst.ud_morph_reduced[i-1].text +" "+ sona.text + " " + tekst.ud_morph_reduced[i+1].text)
            #print("\n" + sona.text + ", vorm: " + sona.form[0] + ", pos: " + sona.pos[0])
            posid.append(sona.pos[0])
            for p in vormid:
                valed.append(p)
                print("automaat: " + p)

valed_dict = Counter(valed)
valed_sorted = valed_dict.most_common(5)
for a in valed_sorted:
    print(a)
    
print("\nPosid")
p_dict = Counter(posid)
p_sorted = p_dict.most_common(5)
for a in p_sorted:
    print(a)


. 1994 toimus
automaat: ?

Need 18 kirja
automaat: ?

" 100 maailma
automaat: ?

" 100 maailma
automaat: ?

umbes 8 tundi
automaat: ?

mängufilmid 1982 "
automaat: ?

" 1986 "
automaat: ?

" 1987 "
automaat: ?

" 1988 "
automaat: ?

" 1989 "
automaat: ?

" 1991 "
automaat: ?

" 1993 "
automaat: ?

" 1996 "
automaat: ?

" 1999 "
automaat: ?

8 1/2 naist
automaat: ?

kell 20.00 esietendub
automaat: ?

valitud 120 kunstnikku
automaat: ?

( 31 )
automaat: ?

( 1989 )
automaat: ?

vormis Maile Grünberg
automaat: sg all
automaat: pl all
automaat: sg all
automaat: pl p

Kell 11.00 saabus
automaat: ?

Kell 20.00 algas
automaat: ?

ajakiri Art lubas
automaat: ?

; VALT ERNSTREIT
automaat: ?

TOOMAS HELP --
automaat: ?

( 1991 )
automaat: ?

tähistas 10 aastat
automaat: ?

isegi igasugu Ivan
automaat: 

regiooni 6500 sõdurit
automaat: ?

ja 80 lennukit
automaat: ?

Missioon 2007 ning
automaat: ?

kuupäeva 1. oktoober
automaat: ?

kaevandusväljalt 16 puutüve
automaat: ?

on 2-3 meetrit
automaat:


Soomest 100 ja
automaat: ?

Rootsist 150 aastat
automaat: ?

umbes 380 sealset
automaat: ?

keskeltläbi 150 marka
automaat: ?

Eestis 200 krooni
automaat: ?

umbes 30 aastat
automaat: ?

umbes 300 krooni
automaat: ?

Don Migueli veelgi
automaat: adt
automaat: sg g
automaat: sg p

11 700 pudelit
automaat: ?

( 14 %
automaat: ?

( 1986 )
automaat: ?

Luuletused 1987--1991 "
automaat: ?

( 1995 )
automaat: ?

( 1997 )
automaat: ?

lk 34 )
automaat: ?

lk 49 )
automaat: ?

annan 4 dollarit
automaat: ?

ja 35 senti
automaat: ?

lk 46 )
automaat: ?

lk 47 )
automaat: ?

lk 29 )
automaat: ?

lk 40 )
automaat: ?

lk 20 ,
automaat: ?

Best New Band
automaat: ?

et 20 aastat
automaat: ?

aastal 2020 Suede'i
automaat: ?

MORNA ABC "
automaat: ?

, 1982 )
automaat: ?

popgrupi ABC juhtlaulja
automaat: ?

Lilac Time "
automaat: pl p

Lilac Time oli
automaat: pl p

Lilac Time .
automaat: pl p

Lilac Time seekord
automaat: pl p

Is Normal "
automaat: sg ad

ja Janet Jacksoni
automaat: sg p

juhi Tap


Nestlé 495 vabrikut
automaat: ?

, 10 kilo
automaat: ?

, 15. mai
automaat: ?

( 27 )
automaat: ?

, 16. mai
automaat: ?

, 17. mai
automaat: ?

, 18. mai
automaat: ?

( 25 )
automaat: ?

kuus 4000 krooni
automaat: ?

Aastatel 1994--1997 õppis
automaat: ?

eestlastel omad rahvatantsud
automaat: pl n
automaat: pl n
automaat: pl n
automaat: d

venelastel omad ,
automaat: pl n
automaat: pl n
automaat: pl n
automaat: d

eesti omad paremad
automaat: pl n
automaat: pl n
automaat: pl n
automaat: d

kulus 2358 krooni
automaat: ?

vaadanud 1300 inimest
automaat: ?

härra Q "
automaat: ?

vaid 30 protsenti
automaat: ?

. EE püüab
automaat: 
automaat: sg g
automaat: sg g

Autowebi omad on
automaat: pl n
automaat: pl n
automaat: pl n
automaat: d

Mardot 20 aastat
automaat: ?

silmapaistev malevaist ,
automaat: pl el

kabel 1935/1936 (
automaat: ?

mälestusmärk 1984 (
automaat: ?

kvartalis 20 võivad
automaat: ?

kvartal 20 on
automaat: ?

kvartal 20 katab
automaat: ?

umbes 800 ruutmeetrit
automa

, La Clusaz's
automaat: ?

Prantsusmaal La Clusaz's
automaat: ?

kahekordne kuldmedalist Kristina
automaat: sg el

( 21 )
automaat: ?

peatreener Franz Gattermann
automaat: ?

. Franz Gattermann
automaat: ?

Azovstalile 1 :
automaat: ?

: 8 tekkisid
automaat: ?

ette 21 :
automaat: ?

: 24 andis
automaat: ?

olete nudist või
automaat: sg el

kuni 50 %
automaat: ?

- 120 grammi
automaat: ?

ühest kolmeni .
automaat: sg ter

( 1929-2006 )
automaat: ?

" Tagurpidi "
automaat: 

. INDIE-ROKK Babyshambles
automaat: ?

BLINDING EP "
automaat: 

aastal 2003 ,
automaat: ?

Sunday New York
automaat: ?

koguni 18 %
automaat: ?

vaid 13,5 %
automaat: ?

on 26 %
automaat: ?

juba 30 aastat
automaat: ?

koguni 23 (
automaat: ?

Suure 1776. aastal
automaat: ?

Ligi 800 aastat
automaat: ?

100 000 eurot
automaat: ?

Aastaks 2040 jääb
automaat: ?

varastatud 221 eset
automaat: ?

kui 5000 aastat
automaat: ?

asus 1955 taas
automaat: ?

( 22 )
automaat: ?

kell 16 avatakse
automaat: ?

välja 100 aastat

automaat: ?

kuni 150 enam
automaat: ?

. 27 eestimaalast
automaat: ?

teedel 100-150 rohkem
automaat: ?

tervelt 721 inimest
automaat: ?

aastaga 20 protsenti
automaat: ?

kõigest 60 krooni
automaat: ?

umbes 1,4-1,7 korda
automaat: ?

tekkima 40-50 tonni
automaat: ?

- 5,1 .
automaat: ?

keskmiselt 6,2 päeva
automaat: ?

olema 1,5-2 korda
automaat: ?

palgast 1,8 korda
automaat: ?

keskmiselt 5300 ,
automaat: ?

tohter 9500 krooni
automaat: ?

haiglates 4191 inimest
automaat: ?

on 721 neist
automaat: ?

tervelt 461 senist
automaat: ?

kasvab 20 %
automaat: ?

kasvab 10 %
automaat: ?

kohta 1966 .
automaat: ?

. 1991 võeti
automaat: ?

, 1994 võeti
automaat: ?

Luup 16.9.2000 )
automaat: ?

juba 29 maad
automaat: ?

sotsioloog Marti Taru
automaat: adt
automaat: sg p
automaat: adt
automaat: sg p

364 914 valimisealist
automaat: ?

vaid 3 %
automaat: ?

ja 28 %
automaat: ?

§ 62 muutmist
automaat: ?

mittevalijatest 31 %
automaat: ?

oli 61 %
automaat: ?

arvamusel 52 %
automaat: ?

. 

...

[ 20 ]
automaat: ?

ajavahemikus 1997-2005 jõudnud
automaat: ?

aastateks 2006-2009 (
automaat: ?

[ 1 ]
automaat: ?

[ 6 ]
automaat: ?

kui 100 koolis
automaat: ?

[ 1 ]
automaat: ?

[ 2 ]
automaat: ?

[ 3 ]
automaat: ?

[ 4 ]
automaat: ?

[ 5 ]
automaat: ?

on 31. märts
automaat: ?

märts 2008 ,
automaat: ?

kestus 30 kuud
automaat: ?

osaleb 17 erinevat
automaat: ?

oktoober 2005 -
automaat: ?

november 2005 )
automaat: ?

oktoober 2005 -
automaat: ?

september 2006 )
automaat: ?

) 1. versioon
automaat: ?

töövahendite 1. versioon
automaat: ?

oktoober 2006 -
automaat: ?

märts 2007 )
automaat: ?

Versioonile 2 on
automaat: ?

oktoober 2006 -
automaat: ?

september 2007 )
automaat: ?

Versioonile 3 on
automaat: ?

ToolBox 2. versioon
automaat: ?

- 15 )
automaat: ?

jaanuar 2006 )
automaat: ?

september 2006 )
automaat: ?

ToolBox 1. versioon
automaat: ?

) 3. versioon
automaat: ?

mai 2007 )
automaat: ?

ToolBox 2. versioon
automaat: ?

) 4. versioon
automaat: ?

september 2007 )



7,4 % rasvhapete
automaat: ?

, 1996 )
automaat: ?

, 1994 )
automaat: ?

, 1991 ;
automaat: ?

, 1993 )
automaat: ?

E. van Elswyki
automaat: ?

( 1997 )
automaat: ?

( 1990 )
automaat: ?

8,9 %-ni ,
automaat: sg ter

sisaldas 20 %
automaat: ?

, 1992 )
automaat: ?

. 1995 .
automaat: ?

, 1995 )
automaat: ?

( 1995 )
automaat: ?

, 1996 )
automaat: ?

, 1996 )
automaat: ?

E. van Elswyki
automaat: ?

( 1995 )
automaat: ?

, van Elswyk
automaat: ?

, % (
automaat: ?

, van Elswyk
automaat: ?

, 1995 )
automaat: ?

tabelist 22 nähtub
automaat: ?

( 1996 )
automaat: ?

, 1998 )
automaat: ?

Tabelist 23 on
automaat: ?

, 1998 )
automaat: ?

, 1994 )
automaat: ?

et 20 %
automaat: ?

( 1991 )
automaat: ?

, 1972 )
automaat: ?

, 1994 )
automaat: ?

oli 1 %
automaat: ?

, 1994 )
automaat: ?

ja 0,92 .
automaat: ?

3,5 % )
automaat: ?

, 1997 )
automaat: ?

Tabelis 24 esitatud
automaat: ?

erineda 73 korda
automaat: ?

( 1997 )
automaat: ?

, 1997 )
automaat: ?

; 1997 )
automaat: ?

, 199


osales 11 riiki
automaat: ?

aastal George Dobsoni
automaat: pl p

satelliidid Explorer-7 (
automaat: ?

( 1959 )
automaat: ?

ja TIROS-1 (
automaat: ?

( 1960 )
automaat: ?

Projekt 1971-1980 ,
automaat: ?

Dekaad 1965-1974 ,
automaat: ?

Programm 1981-1990 ,
automaat: ?

olemas 7 selle
automaat: ?

kalendrites 5 aastat
automaat: ?

ja 3 aastat
automaat: ?

jumalaks Samas ,
automaat: sg in
automaat: 

saadi 365 päeva
automaat: ?

kokku 27 sfääri
automaat: ?

arvu 34-ni ja
automaat: sg ter

veel 22 sfääri
automaat: ?

juba 56 sfääri
automaat: ?

( 87-165 )
automaat: ?

( 1473-1543 )
automaat: ?

( 1546-1601 )
automaat: ?

( 1571-1630 )
automaat: ?

( 1564-1642 )
automaat: ?

( 1629-1695 )
automaat: ?

( 1643-1727 )
automaat: ?

( 1900-1992 )
automaat: ?

( 1905-1973 )
automaat: ?

( 1656-1742 )
automaat: ?

avastatud 3 päris
automaat: ?

( 1781 )
automaat: ?

( 1846 )
automaat: ?

( 1930 )
automaat: ?

( 1977 )
automaat: ?

siiski 7 kraadi
automaat: ?

teadlane George Louis
automaat: 


. Rokib :
automaat: b

. miilis :
automaat: sg in

? kris0007 :
automaat: ?

vähemalt 10 punni
automaat: ?

keel 8 +
automaat: ?

keel 8 +
automaat: ?

: Ahah .
automaat: 

10-klassi Posti0 :
automaat: ?

jah kris0007 :
automaat: ?

( 36 õpilast
automaat: ?

aga 3 tykki
automaat: ?

" agateatemis "
automaat: sg in
automaat: sg in

mingi 7.-8. klass
automaat: ?

on mata alati
automaat: o

alati 5 olnud
automaat: ?

: füüsima matemaatika
automaat: sg g

on 6 ja
automaat: ?

. dgk :
automaat: ?

nädalas 5 matet
automaat: ?

, 3 füüsikat
automaat: ?

ja 3 keemiat
automaat: ?

keemiat sannu7 :
automaat: ?

arvust 256 näiteks
automaat: ?

arv 2 Jagad
automaat: ?

Kuna 1 ei
automaat: ?

arv 105 kahendsüsteemis
automaat: ?

on 100 v
automaat: ?

. Zeppelist :
automaat: sg el
automaat: sg el
automaat: sg p

. Geenius666 :
automaat: ?

... ehee ...
automaat: 

12 kl 3d
automaat: ?

: PC_man :
automaat: 
automaat: 

: TÖNA ?
automaat: ?

:S PC_man :
automaat: 
automaat: 

kodukal Ceus12345 :
aut


. dgk :
automaat: ?

. s33maNN :
automaat: ?

küll ale-hopp asendada
automaat: 

on 15 õlut
automaat: ?

ja 10 Sakut
automaat: ?

ja hää .
automaat: 

saab 2 originaali
automaat: ?

. MoX :
automaat: ?

umbes 50-80 %
automaat: ?

? d2xz :
automaat: ?

või tatra (
automaat: sg g

mis veel ...
automaat: 
automaat: sg ad

:S crash_343 :
automaat: ?

) markko666 :
automaat: ?

Porsche 356 pepu
automaat: ?

kuna 99.9 %
automaat: ?

" n6u offence
automaat: ?

üleüldse kehvavõitu .
automaat: 
automaat: 

rohkem DMX :
automaat: ?

128A d2xz :
automaat: ?

toodab markko666 :
automaat: ?

et tipp-topp porche
automaat: 

: k6ik on
automaat: ?

... d2xz :
automaat: ?

uus 700 ne
automaat: ?

tulesid d2xz :
automaat: ?

vähemalt 35 aastat
automaat: ?

mitte 25 nagu
automaat: ?

tegema d2xz :
automaat: ?

leida crash_343 :
automaat: ?

üks bmw (
automaat: ?

: 300ne bmw
automaat: sg g

300ne bmw on
automaat: ?

. d2xz :
automaat: ?

on 300ne .
automaat: sg g

. d2xz :
automaat: ?

: 300ne põmm
auto

### Mis sõnaliigis sõnad mis vale vormi analüüsid said 

In [121]:
valed = []
arve = 0
for tekst in kogu_korpus:
    i = -1 
    for sona in tekst.ud_morph_reduced:
        i += 1
        if sona.pos[0] == 'N':
            arve += 1
            vormid = tekst.morph_oletamistega[i].form
            for v in vormid:
                valed.append(v)
                
valed_dict = Counter(valed)
valed_sorted = valed_dict.most_common()
print(arve)
for a in valed_sorted:
    print(a)

10328
('?', 6747)
('sg n', 2471)
('sg g', 1455)
('sg p', 936)
('adt', 439)
('', 272)
('sg el', 154)
('sg ad', 139)
('sg in', 132)
('pl p', 132)
('s', 105)
('o', 82)
('sg tr', 74)
('sg all', 73)
('pl n', 68)
('pl g', 36)
('sg kom', 28)
('sg ter', 19)
('sg abl', 15)
('pl ad', 10)
('d', 7)
('pl el', 7)
('sg es', 6)
('pl in', 6)
('pl all', 4)
('ks', 4)
('pl ill', 3)
('sg ill', 3)
('te', 1)
('pl kom', 1)
('des', 1)
('pl ter', 1)


### Ühestamisvead

#### Lemmade ühestamisvead sõnaliigiti

In [51]:
valed = []
posid = []
mata = 0
mata2 = 0
for tekst in loaded_texts_edt_am:
    i = -1 
    for sona in tekst.ud_morph_reduced:
        i += 1
        vormid = tekst.morph_yhestamisega[i].partofspeech
        rootid_yh = eemalda_jar(tekst.morph_yhestamisega[i].annotations, alakriips = True)
        rootid_ol = eemalda_jar(tekst.morph_oletamistega[i].annotations, alakriips = True)
        if sona.lemma[0] not in rootid_yh and sona.lemma[0] in rootid_ol:
            #print("\n" + tekst.ud_morph_reduced[i-1].text +" "+ sona.text + " " + tekst.ud_morph_reduced[i+1].text)
            #print("\n" + sona.text + ", vorm: " + sona.form[0] + ", pos: " + sona.pos[0])
            posid.append(sona.pos[0])
            for p in vormid:
                valed.append(p)
                #print("automaat: " + p)

valed_dict = Counter(valed)
valed_sorted = valed_dict.most_common(5)
print("Vale")
for a in valed_sorted:
    print(a)
    
print("\nKuld")
p_dict = Counter(posid)
p_sorted = p_dict.most_common(5)
for a in p_sorted:
    print(a)
    

Vale
('S', 3725)
('H', 2181)
('P', 720)
('V', 549)
('A', 522)

Kuld
('H', 3188)
('S', 1544)
('D', 706)
('A', 697)
('P', 669)


#### Spetsiifilise sõnaliigi valede analüüside kontrollimine

In [55]:
valed = []
posid = []
mata = 0
mata2 = 0
for tekst in loaded_texts_edt_am:
    i = -1 
    for sona in tekst.ud_morph_reduced:
        i += 1
        vormid = tekst.morph_yhestamisega[i].partofspeech
        vormid_ol = tekst.morph_oletamistega[i].partofspeech
        if sona.pos[0] not in vormid and sona.pos[0] in vormid_ol and sona.pos[0] == 'S':
            #print("\n" + tekst.ud_morph_reduced[i-1].text +" "+ sona.text + " " + tekst.ud_morph_reduced[i+1].text)
            #print("\n" + sona.text + ", vorm: " + sona.form[0] + ", pos: " + sona.pos[0])
            posid.append(sona.pos[0])
            for p in vormid:
                valed.append(p)
                #print("automaat: " + p)

valed_dict = Counter(valed)
valed_sorted = valed_dict.most_common(5)
print("Vale")
for a in valed_sorted:
    print(a)
    
print("\nKuld")
p_dict = Counter(posid)
p_sorted = p_dict.most_common(5)
for a in p_sorted:
    print(a)

Vale
('H', 1008)
('A', 505)
('V', 259)
('K', 173)
('D', 116)

Kuld
('S', 1788)


### Mis sõnaliigid kõige tihemini vale analüüsi said

In [25]:
print("EDT")
vigu_liik(loaded_texts_edt_am)
print("\nEWT")
vigu_liik(loaded_texts_ewt_am)
print("\nKogu koprus")
vigu_liik(kogu_korpus)

EDT

Normaliseeritud: 
D: 2.74
Z: 0.94
X: 99.85
S: 0.42
P: 1.19
H: 1.03
Y: 4.33
A: 0.52
N: 0.7
V: 0.07
G: 5.0
O: 1.15
I: 6.54
K: 0.15
J: 0.02
C: 0.04

Normaliseerimata: 
D: 0.26%
Z: 0.15%
X: 0.15%
S: 0.11%
P: 0.08%
H: 0.06%
Y: 0.04%
A: 0.04%
N: 0.01%
V: 0.01%
G: 0.01%
O: 0.01%
I: 0.0%
K: 0.0%
J: 0.0%
C: 0.0%

EWT

Normaliseeritud: 
H: 36.1
D: 4.63
Z: 1.84
X: 100.0
S: 1.32
V: 0.98
A: 3.91
I: 34.02
Y: 16.58
P: 1.73
N: 2.59
J: 0.58
O: 8.37
K: 1.24
C: 1.52

Normaliseerimata: 
H: 1.53%
D: 0.62%
Z: 0.31%
X: 0.25%
S: 0.25%
V: 0.18%
A: 0.18%
I: 0.18%
Y: 0.18%
P: 0.18%
N: 0.04%
J: 0.04%
O: 0.03%
K: 0.02%
C: 0.01%

Kogu koprus

Normaliseeritud: 
D: 3.1
H: 4.99
Z: 1.07
X: 99.88
S: 0.52
P: 1.3
Y: 6.37
A: 0.87
V: 0.22
I: 21.58
N: 0.91
O: 1.81
J: 0.12
G: 4.64
K: 0.27
C: 0.21

Normaliseerimata: 
D: 0.31%
H: 0.27%
Z: 0.18%
X: 0.17%
S: 0.13%
P: 0.1%
Y: 0.06%
A: 0.06%
V: 0.04%
I: 0.03%
N: 0.02%
O: 0.01%
J: 0.01%
G: 0.01%
K: 0.01%
C: 0.0%


### Näidete otsimine

In [88]:
valed = []
for tekst in kogu_korpus:
    i = -1 
    for sona in tekst.ud_morph_reduced:
        i += 1
        posid = tekst.morph_oletamistega[i].partofspeech
        if sona.pos[0] not in posid and 'Y' == sona.pos[0] and 'S' in posid and 'H' not in posid:
            #print("\n" + tekst.ud_morph_reduced[i-1].text +" "+ sona.text + " " + tekst.ud_morph_reduced[i+1].text)
            print("\n" + sona.text + ", pos: " + sona.pos[0])
            for p in posid:
                valed.append(p)
                print("automaat: " + p)

valed_dict = Counter(valed)
valed_sorted = valed_dict.most_common()
for a in valed_sorted:
    print(a)


WiFi, pos: Y
automaat: S
automaat: S

WiFi, pos: Y
automaat: S
automaat: S

relig., pos: Y
automaat: S

ibid, pos: Y
automaat: S
automaat: V
automaat: S

ibid, pos: Y
automaat: S
automaat: V
automaat: S

ibid, pos: Y
automaat: S
automaat: V
automaat: S

ibid, pos: Y
automaat: S
automaat: V
automaat: S

ibid, pos: Y
automaat: S
automaat: V
automaat: S

toim, pos: Y
automaat: S

toim., pos: Y
automaat: S

.ee, pos: Y
automaat: I
automaat: S

IgE, pos: Y
automaat: S

IgA, pos: Y
automaat: P
automaat: P
automaat: P
automaat: S
automaat: S

IgA, pos: Y
automaat: P
automaat: P
automaat: P
automaat: S
automaat: S

IgE, pos: Y
automaat: S

IgE, pos: Y
automaat: S

IgE, pos: Y
automaat: S

IgE, pos: Y
automaat: S

ee, pos: Y
automaat: I
automaat: S

etc, pos: Y
automaat: S

orig., pos: Y
automaat: S

plüüssis, pos: Y
automaat: S
automaat: S
automaat: V
automaat: S

10kilo, pos: Y
automaat: S
automaat: S
automaat: S

umb, pos: Y
automaat: S

hyphenator.js, pos: Y
automaat: S

et.js, pos: Y
auto

### Liitsõnade vead

In [14]:
liitsona(kogu_korpus)

Kuld: Laas_mäe, automaatne: Laasmäe
Kuld: Laas_mäe, automaatne: Laasmäe
Kuld: Toom_pere, automaatne: Toompere
Kuld: Puustus_maa, automaatne: Puustusmaa
Kuld: Heina_puu, automaatne: Heinapuu
Kuld: Risti_kivi, automaatne: Ristikivi
Kuld: Sini_järv, automaatne: Sinijärv
Kuld: Sini_järv, automaatne: Sinijärv
Kuld: Ida-Saksa_maa, automaatne: Ida-Saksamaa
Kuld: Ida-Saksa_maa, automaatne: Ida-Saksamaa
Kuld: Ida-Saksa_maa, automaatne: Ida-Saksamaa
Kuld: Ida-Saksa_maa, automaatne: Ida-Saksamaa
Kuld: Lääne-Viru_maa, automaatne: Lääne-Virumaa
Kuld: Lääne-Viru_maa, automaatne: Lääne-Virumaa
Kuld: Küla_ots, automaatne: Külaots
Kuld: Nõu_kogu, automaatne: Nõukogu
Kuld: Vaba_riik, automaatne: Vabariik
Kuld: Riigi_kogu, automaatne: Riigikogu
Kuld: kiir_prohhodka, automaatne: kiirprohhodka
Kuld: Hansa_pank, automaatne: Hansapank
Kuld: Hansa_pank, automaatne: Hansapank
Kuld: Hansa_pank, automaatne: Hansapank
Kuld: rahvus_vahelistamine, automaatne: rahvusvahelistamine
Kuld: Vilu_oja, automaatne: Viluoja


Kuld: Suur_saar, automaatne: Suursaar
Kuld: Vald_mann, automaatne: Valdmann
Kuld: Roht_mets, automaatne: Rohtmets
Kuld: Nõu_kogu, automaatne: Nõukogu
Kuld: Vene_maa-Tšetšeenia, automaatne: Venemaa-Tšetšeenia
Kuld: Kure_soo, automaatne: Kuresoo
Kuld: Somaali_maa, automaatne: Somaalimaa
Kuld: Tamm_saar, automaatne: Tammsaar
Kuld: Hansa_pank, automaatne: Hansapank
Kuld: Kivi_saar, automaatne: Kivisaar
Kuld: Rand_pere, automaatne: Randpere
Kuld: Rand_pere, automaatne: Randpere
Kuld: Nõu_kogu, automaatne: Nõukogu
Kuld: Laan_järv, automaatne: Laanjärv
Kuld: Laan_järv, automaatne: Laanjärv
Kuld: Oja_soon, automaatne: Ojasoon
Kuld: Nõu_kogu, automaatne: Nõukogu
Kuld: Nõu_kogu, automaatne: Nõukogu
Kuld: Riigi_kogu, automaatne: Riigikogu
Kuld: Tamme_mäe, automaatne: Tammemäe
Kuld: Nõu_kogu, automaatne: Nõukogu
Kuld: Kirsi_puu, automaatne: Kirsipuu
Kuld: Uue_küla, automaatne: Uueküla
Kuld: Uue_küla, automaatne: Uueküla
Kuld: Ida-Viru_maa, automaatne: Ida-Virumaa
Kuld: sula_saksa, automaatne: sula

Kuld: linoleen_happe_sisaldus, automaatne: linoleenhappesisaldus
Kuld: linoleen_happe_sisaldus, automaatne: linoleenhappesisaldus
Kuld: amüloi_doos, automaatne: amüloidoos
Kuld: amüloi_doos, automaatne: amüloidoos
Kuld: ekstra_tsellulaarne, automaatne: ekstratsellulaarne
Kuld: retiku_liin, automaatne: retikuliin
Kuld: glükoos_amino_glükaan, automaatne: glükoosaminoglükaan
Kuld: mitte_fibrillaarne, automaatne: mittefibrillaarne
Kuld: mitte_fibrillaarne, automaatne: mittefibrillaarne
Kuld: auto_agregatsioon, automaatne: autoagregatsioon
Kuld: amüloidi_fibrill, automaatne: amüloidifibrill
Kuld: radio_märgistatud, automaatne: radiomärgistatud
Kuld: radio_märgistatud, automaatne: radiomärgistatud
Kuld: radio_märgistatud, automaatne: radiomärgistatud
Kuld: amüloi_doos, automaatne: amüloidoos
Kuld: amüloi_doos, automaatne: amüloidoos
Kuld: pingutus_düspnoe, automaatne: pingutusdüspnoe
Kuld: trahheo_bronhopaatia, automaatne: trahheobronhopaatia
Kuld: amüloid_lümf_adenopaatia, automaatne: amülo

Kuld: stiili_freak, automaatne: stiilifreak
Kuld: ümber_sõnasta, automaatne: ümbersõnasta
Kuld: kuuri_tuuning, automaatne: kuurituuning
Kuld: ei_no, automaatne: eino
Kuld: porno_tuuning, automaatne: pornotuuning
Kuld: haha_hahha_haha_haha, automaatne: hahahahhahahahaha
Täpselt sama lemma, alakriipsud samas kohas: 92.82%
Sama lemma, aga alakriipsud eri kohtades: 0.15%
Ilma kriipsu(de)ta, aga sama lemma: 1.35%
Erinev lemma, kriipsudega või mitte : 5.68%
Pole liitsõnana analüüsitud, aga sama lemma: 651
Pärisnimesid: 406, nimisõnu: 169


# Kontrollid

### Kirjavahemärkide arv korpuses

In [12]:
loend = 0
sonu = 0
for tekst in kogu_korpus:
    for sona in tekst.ud_morph_reduced:
        sonu += 1
        if 'Z' in sona.pos:
            loend += 1
print("Kirjavahemärke: " + str(loend))
print(round(loend/sonu*100, 2))

Kirjavahemärke: 82934
16.53


### Emotikonide kontrollimine

In [75]:
loend = 0
for tekst in loaded_texts_ewt_am:
    i = -1
    for sona in tekst.ud_morph_reduced:
        i += 1
        if sona.lemma[0] == ':-(':
            print(sona.pos[0], tekst.morph_oletamistega[i].partofspeech)

### Kas on mitmesi analüüse, mille üks osa on None

In [39]:
loend = 0
for tekst in kogu_korpus:
    for sona in tekst.morph_oletamisteta:
        if None in sona.partofspeech and len(sona.partofspeech) > 1:
            loend += 1
print(loend)

0


### Ks lühike ja pikk vorm kontroll

In [42]:
for tekst in kogu_korpus:
    for sona in tekst.ud_morph_reduced:
        if re.match('ksin|ksid|ksime|ksite', sona.form[0]):
            print(sona)

Span('tahaksin', [{'lemma': 'taht', 'pos': 'V', 'form': 'ksin'}])
Span('tahaksin', [{'lemma': 'taht', 'pos': 'V', 'form': 'ksin'}])
Span('peaksin', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksin'}])
Span('tahaksite', [{'lemma': 'taht', 'pos': 'V', 'form': 'ksite'}])
Span('Võiksin', [{'lemma': 'või', 'pos': 'V', 'form': 'ksin'}])
Span('rõhutaksin', [{'lemma': 'rõhuta', 'pos': 'V', 'form': 'ksin'}])
Span('saaksid', [{'lemma': 'saa', 'pos': 'V', 'form': 'ksid'}])
Span('jääksid', [{'lemma': 'jää', 'pos': 'V', 'form': 'ksid'}])
Span('ütleksin', [{'lemma': 'ütle', 'pos': 'V', 'form': 'ksin'}])
Span('pakuksin', [{'lemma': 'pakku', 'pos': 'V', 'form': 'ksin'}])
Span('võiksid', [{'lemma': 'või', 'pos': 'V', 'form': 'ksid'}])
Span('leiaksid', [{'lemma': 'leid', 'pos': 'V', 'form': 'ksid'}])
Span('peaksid', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksid'}])
Span('müüksid', [{'lemma': 'müü', 'pos': 'V', 'form': 'ksid'}])
Span('peaksid', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksid'}])
Span('oleksid', 

Span('peaksid', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksid'}])
Span('tuleksin', [{'lemma': 'tule', 'pos': 'V', 'form': 'ksin'}])
Span('vaataksin', [{'lemma': 'vaata', 'pos': 'V', 'form': 'ksin'}])
Span('oleksid', [{'lemma': 'ole', 'pos': 'V', 'form': 'ksid'}])
Span('tahaksid', [{'lemma': 'taht', 'pos': 'V', 'form': 'ksid'}])
Span('täidaksid', [{'lemma': 'täit', 'pos': 'V', 'form': 'ksid'}])
Span('peaksid', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksid'}])
Span('võiksid', [{'lemma': 'või', 'pos': 'V', 'form': 'ksid'}])
Span('võiksid', [{'lemma': 'või', 'pos': 'V', 'form': 'ksid'}])
Span('näeksite', [{'lemma': 'näge', 'pos': 'V', 'form': 'ksite'}])
Span('oleksid', [{'lemma': 'ole', 'pos': 'V', 'form': 'ksid'}])
Span('peaksime', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksime'}])
Span('teeniksite', [{'lemma': 'teeni', 'pos': 'V', 'form': 'ksite'}])
Span('tahaksite', [{'lemma': 'taht', 'pos': 'V', 'form': 'ksite'}])
Span('hakkaksin', [{'lemma': 'hakka', 'pos': 'V', 'form': 'ksin'}])
Span('v

Span('leiaksid', [{'lemma': 'leid', 'pos': 'V', 'form': 'ksid'}])
Span('veenduksid', [{'lemma': 'veendu', 'pos': 'V', 'form': 'ksid'}])
Span('sammuksid', [{'lemma': 'sammu', 'pos': 'V', 'form': 'ksid'}])
Span('näeksid', [{'lemma': 'näge', 'pos': 'V', 'form': 'ksid'}])
Span('räägiksid', [{'lemma': 'rääki', 'pos': 'V', 'form': 'ksid'}])
Span('oleksid', [{'lemma': 'ole', 'pos': 'V', 'form': 'ksid'}])
Span('ilmuksid', [{'lemma': 'ilmu', 'pos': 'V', 'form': 'ksid'}])
Span('oleksid', [{'lemma': 'ole', 'pos': 'V', 'form': 'ksid'}])
Span('tuleksid', [{'lemma': 'tule', 'pos': 'V', 'form': 'ksid'}])
Span('muneksid', [{'lemma': 'mune', 'pos': 'V', 'form': 'ksid'}])
Span('laseksid', [{'lemma': 'lask', 'pos': 'V', 'form': 'ksid'}])
Span('peaksin', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksin'}])
Span('oleksid', [{'lemma': 'ole', 'pos': 'V', 'form': 'ksid'}])
Span('saaksin', [{'lemma': 'saa', 'pos': 'V', 'form': 'ksin'}])
Span('peaksid', [{'lemma': 'pida', 'pos': 'V', 'form': 'ksid'}])
Span('Tahaksi

# Statistika

### Üldstatistika

In [69]:
print("EDT korpuse üldstatistika")
yldstatistika(edt_korpusepohistega)
print("\nAjakirjandustekstide statistika")
yldstatistika(aja)
print("\nIlukirjandustekstide statistika")
yldstatistika(ilu)
print("\nTeadustekstide statistika")
yldstatistika(tea)
print("\nEWT korpuse üldstatistika")
yldstatistika(ewt_korpusepohistega)
print("\nMääramatute netitekstide statistika")
yldstatistika(ewtb)
print("\nFoorumite statistika")
yldstatistika(foorum)
print("\nKommentaariumite statistika")
yldstatistika(kom)
print("\nMõlema korpuse peale kokku statistika")
yldstatistika(kogu_korpus)

EDT korpuse üldstatistika
Lauseid:                 29695
Sõnu:                    428319
Sõnu kirjavahemärkideta: 357737
Dokumente:               36

Ajakirjandustekstide statistika
Lauseid:                 18690
Sõnu:                    266864
Sõnu kirjavahemärkideta: 224762
Dokumente:               19

Ilukirjandustekstide statistika
Lauseid:                 5522
Sõnu:                    67695
Sõnu kirjavahemärkideta: 54225
Dokumente:               10

Teadustekstide statistika
Lauseid:                 5483
Sõnu:                    93760
Sõnu kirjavahemärkideta: 78750
Dokumente:               7

EWT korpuse üldstatistika
Lauseid:                 5863
Sõnu:                    72698
Sõnu kirjavahemärkideta: 60346
Dokumente:               41

Määramatute netitekstide statistika
Lauseid:                 1715
Sõnu:                    27304
Sõnu kirjavahemärkideta: 23056
Dokumente:               32

Foorumite statistika
Lauseid:                 2783
Sõnu:                    32698
Sõnu kirj

### Tundmatute sõnade statistika

In [13]:
print("EDT korpuse tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(loaded_texts_edt_am)
print("\nAjakirjandustekstide statistika")
print("---------------------------------------------------------")
tundmatud(aja)
print("\nIlukirjandustekstide statistika")
print("---------------------------------------------------------")
tundmatud(ilu)
print("\nTeadustekstide statistika")
print("---------------------------------------------------------")
tundmatud(tea)
print("\nEWT korpuse tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(loaded_texts_ewt_am)
print("\nMääramatute netitekstide statistika")
print("---------------------------------------------------------")
tundmatud(ewtb)
print("\nFoorumite statistika")
print("---------------------------------------------------------")
tundmatud(foorum)
print("\nKommentaariumite statistika")
print("---------------------------------------------------------")
tundmatud(kom)
print("\nMõlema korpuse peale kokku tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(kogu_korpus)

EDT korpuse tundmatute sõnade statistika
---------------------------------------------------------
VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta
Tundmatuid sõnu: 80678 ehk 18.84% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 48094 ehk 13.44% sõnadest

VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta, aga slängisõnastikuga
Tundmatuid sõnu: 80657 ehk 18.83% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 48075 ehk 13.44% sõnadest

Ajakirjandustekstide statistika
---------------------------------------------------------
VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta
Tundmatuid sõnu: 48457 ehk 18.16% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 31132 ehk 13.85% sõnadest

VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta, aga slängisõnastikuga
Tundmatuid sõnu: 48445 ehk 18.15% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 31121 ehk 13.85% sõnadest

Ilukirjandustekstide statistika
---------------------------------------------------------
VabamorfAnalyzer il

### Tundmatute sagedus sõnaliigiti

### Kirjavahemärkideta

In [10]:
print("EDT korpuse tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(loaded_texts_edt_am)
print("\nAjakirjandustekstide statistika")
print("---------------------------------------------------------")
tundmatud(aja)
print("\nIlukirjandustekstide statistika")
print("---------------------------------------------------------")
tundmatud(ilu)
print("\nTeadustekstide statistika")
print("---------------------------------------------------------")
tundmatud(tea)
print("\nEWT korpuse tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(loaded_texts_ewt_am)
print("\nMääramatute netitekstide statistika")
print("---------------------------------------------------------")
tundmatud(ewtb)
print("\nFoorumite statistika")
print("---------------------------------------------------------")
tundmatud(foorum)
print("\nKommentaariumite statistika")
print("---------------------------------------------------------")
tundmatud(kom)
print("\nMõlema korpuse peale kokku tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(kogu_korpus)

EDT korpuse tundmatute sõnade statistika
---------------------------------------------------------
VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta
Tundmatuid sõnu: 80399 ehk 18.77% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 12062 ehk 3.37% sõnadest
H: 64.4%
S: 19.52%
Y: 8.94%
A: 4.64%
D: 0.67%
V: 0.63%
N: 0.5%
I: 0.23%
G: 0.16%
O: 0.12%
P: 0.12%
K: 0.03%
C: 0.02%
U: 0.01%

VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta, aga slängisõnastikuga
Tundmatuid sõnu: 80378 ehk 18.77% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 12041 ehk 3.37% sõnadest
H: 64.5%
S: 19.41%
Y: 8.95%
A: 4.65%
D: 0.66%
V: 0.63%
N: 0.5%
I: 0.23%
G: 0.15%
O: 0.12%
P: 0.12%
K: 0.03%
C: 0.02%
U: 0.01%

Ajakirjandustekstide statistika
---------------------------------------------------------
VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta
Tundmatuid sõnu: 48380 ehk 18.13% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 7432 ehk 3.31% sõnadest
H: 79.0%
S: 11.67%
Y: 6.31%
A: 1.28%
V: 0.59%
D: 

{'Javeelüks',
 'Kellegil',
 'KÕIIK',
 'Mede',
 'Mi-da',
 'Mi-i-i-is',
 'Mi-mina',
 'Mõlemist',
 'asee',
 'en-da-ga',
 'igaksu',
 'iseennnast',
 'issendast',
 'k6igelt',
 'k6ik',
 'k6iki',
 'kellegiga',
 'kellegil',
 'kellegile',
 'koigega',
 'kôik',
 'köik',
 'küik',
 'küikk',
 'm6nes',
 'm6ni',
 'mA',
 'mdagi',
 'migni',
 'milest',
 'millaised',
 'mingel',
 'misket',
 'mislaadi',
 'mittemidagi',
 'mittemillestki',
 'mlle',
 'neeed',
 'neidt',
 'neistsamadest',
 'plju',
 'se',
 'se-da',
 'seed',
 'sellien',
 'selliset',
 'siuksega',
 'siukstega',
 'teienteisega',
 'teistlet',
 'tesitel',
 'to',
 'uksteisele',
 'yhega',
 'yheks',
 'yhele',
 'yhtegi',
 'yks',
 'ykski',
 'üHELT',
 'ühtgi'}

### Normaliseeritud

In [76]:
print("EDT korpuse tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(loaded_texts_edt_am)
print("\nAjakirjandustekstide statistika")
print("---------------------------------------------------------")
tundmatud(aja)
print("\nIlukirjandustekstide statistika")
print("---------------------------------------------------------")
tundmatud(ilu)
print("\nTeadustekstide statistika")
print("---------------------------------------------------------")
tundmatud(tea)
print("\nEWT korpuse tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(loaded_texts_ewt_am)
print("\nMääramatute netitekstide statistika")
print("---------------------------------------------------------")
tundmatud(ewtb)
print("\nFoorumite statistika")
print("---------------------------------------------------------")
tundmatud(foorum)
print("\nKommentaariumite statistika")
print("---------------------------------------------------------")
tundmatud(kom)
print("\nMõlema korpuse peale kokku tundmatute sõnade statistika")
print("---------------------------------------------------------")
tundmatud(kogu_korpus)

EDT korpuse tundmatute sõnade statistika
---------------------------------------------------------
VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta
Tundmatuid sõnu: 80678 ehk 18.84% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 48094 ehk 13.44% sõnadest
S: 12.54%
V: 12.25%
H: 24.36%
D: 11.75%
P: 13.1%
A: 12.11%
J: 14.07%
N: 14.65%
K: 10.66%
Y: 23.32%
O: 14.33%
C: 9.09%
G: 17.5%
I: 21.81%
U: 14.6%

VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta, aga slängisõnastikuga
Tundmatuid sõnu: 80657 ehk 18.83% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 48075 ehk 13.44% sõnadest
S: 12.53%
V: 12.25%
H: 24.36%
D: 11.74%
P: 13.09%
A: 12.11%
J: 14.07%
N: 14.65%
K: 10.66%
Y: 23.32%
O: 14.33%
C: 9.09%
G: 17.34%
I: 21.81%
U: 14.6%

Ajakirjandustekstide statistika
---------------------------------------------------------
VabamorfAnalyzer ilma tavalise ja pärisnimede oletamiseta
Tundmatuid sõnu: 48457 ehk 18.16% sõnadest
Tundmatuid sõnu kirjavahemärkideta: 31132 ehk 13.85% sõnadest
S: 

## Pärisnimede statistika
### Alakriipsuga

In [45]:
print("EDT korpuse pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(loaded_texts_edt_am, alakriips=True)
print("\nAjakirjandustekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(aja, alakriips=True)
print("\nIlukirjandustekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(ilu, alakriips=True)
print("\nTeadustekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(tea, alakriips=True)
print("\nEWT korpuse pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(loaded_texts_ewt_am, alakriips=True)
print("\nMääramatute netitekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(ewtb, alakriips=True)
print("\nFoorumite pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(foorum, alakriips=True)
print("\nKommentaariumite pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(kom, alakriips=True)
print("\nMõlema korpuse peale kokku pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(kogu_korpus, alakriips=True)

EDT korpuse pärisnimede statistika
---------------------------------------------------------
Oletamisteta
Korrektsus: 0.2056126908556657 ehk 20.56%
Saagis: 0.4568783834042729 ehk 45.69%
Täpsus: 0.2721275967313183 ehk 27.21%
F1-skoor: 0.34109244604871425 ehk 34.11%

Pärisnime oletamiseta
Korrektsus: 0.33856021513191364 ehk 33.86%
Saagis: 0.7699904954750196 ehk 77.0%
Täpsus: 0.3766525166767738 ehk 37.67%
F1-skoor: 0.5058572805386254 ehk 50.59%
Oletamistega
Korrektsus: 0.36398043016708204 ehk 36.4%
Saagis: 0.977643704285301 ehk 97.76%
Täpsus: 0.36703538793304064 ehk 36.7%
F1-skoor: 0.5337033026529507 ehk 53.37%

Ajakirjandustekstide pärisnimede statistika
---------------------------------------------------------
Oletamisteta
Korrektsus: 0.21531595019967112 ehk 21.53%
Saagis: 0.4777192891019961 ehk 47.77%
Täpsus: 0.2816061937386709 ehk 28.16%
F1-skoor: 0.35433740528838714 ehk 35.43%

Pärisnime oletamiseta
Korrektsus: 0.3402844468834317 ehk 34.03%
Saagis: 0.7756293323604524 ehk 77.56%
Täpsu

### Alakriipsuta

In [19]:
print("EDT korpuse liitsõnadega arvestamise pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(loaded_texts_edt_am)
print("\nAjakirjandustekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(aja)
print("\nIlukirjandustekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(ilu)
print("\nTeadustekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(tea)
print("\nEWT korpuse liitsõnadega arvestamise pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(loaded_texts_ewt_am)
print("\nMääramatute netitekstide pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(ewtb)
print("\nFoorumite pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(foorum)
print("\nKommentaariumite pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(kom)
print("\nMõlema korpuse peale kokku liitsõnadega arvestamise pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(kogu_korpus)

EDT korpuse liitsõnadega arvestamise pärisnimede statistika
---------------------------------------------------------
Oletamisteta
Korrektsus: 0.206397945511389 ehk 20.64%
Saagis: 0.4583247241621555 ehk 45.83%
Täpsus: 0.27298907157625285 ehk 27.3%
F1-skoor: 0.3421722430468786 ehk 34.22%

Pärisnime oletamiseta
Korrektsus: 0.340191744437774 ehk 34.02%
Saagis: 0.7727592049258234 ehk 77.28%
Täpsus: 0.37800687285223367 ehk 37.8%
F1-skoor: 0.5076762274498092 ehk 50.77%
Oletamistega
Korrektsus: 0.36633480257682705 ehk 36.63%
Saagis: 0.9822719947105252 ehk 98.23%
Täpsus: 0.36877298043657014 ehk 36.88%
F1-skoor: 0.5362299223966793 ehk 53.62%

Ajakirjandustekstide pärisnimede statistika
---------------------------------------------------------
Oletamisteta
Korrektsus: 0.21600150425421896 ehk 21.6%
Saagis: 0.47897013602960337 ehk 47.9%
Täpsus: 0.28234354358044794 ehk 28.23%
F1-skoor: 0.35526519251584976 ehk 35.53%

Pärisnime oletamiseta
Korrektsus: 0.3415728279776618 ehk 34.16%
Saagis: 0.77781831

### Kõige sagedasemad valed vormid

In [34]:
print("EDT korpuse liitsõnadega arvestamise pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(loaded_texts_edt_am)
print("\nEWT korpuse liitsõnadega arvestamise pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(loaded_texts_ewt_am)
print("\nMõlema korpuse peale kokku liitsõnadega arvestamise pärisnimede statistika")
print("---------------------------------------------------------")
parisnimed(kogu_korpus)

EDT korpuse liitsõnadega arvestamise pärisnimede statistika
---------------------------------------------------------
Oletamisteta
Korrektsus: 0.206397945511389 ehk 20.64%
Saagis: 0.4583247241621555 ehk 45.83%
Täpsus: 0.27298907157625285 ehk 27.3%
F1-skoor: 0.3421722430468786 ehk 34.22%

Pärisnime oletamiseta
Korrektsus: 0.340191744437774 ehk 34.02%
Saagis: 0.7727592049258234 ehk 77.28%
Täpsus: 0.37800687285223367 ehk 37.8%
F1-skoor: 0.5076762274498092 ehk 50.77%

Oletamistega
Korrektsus: 0.36633480257682705 ehk 36.63%
Saagis: 0.9822719947105252 ehk 98.23%
Täpsus: 0.36877298043657014 ehk 36.88%
F1-skoor: 0.5362299223966793 ehk 53.62%

Normaliseeritud: 
sg g: 1.97
sg n: 1.35
sg p: 3.95
: 33.33
pl g: 10.43
sg in: 0.8
sg kom: 2.8
sg ad: 1.9
sg all: 1.4
pl n: 9.84
sg abl: 6.0
sg el: 1.24
sg tr: 6.25
sg ill: 0.9
adt: 0.53
pl p: 8.33

Normaliseerimata: 
sg g: 0.67%
sg n: 0.63%
sg p: 0.08%
: 0.07%
pl g: 0.07%
sg in: 0.05%
sg kom: 0.03%
sg ad: 0.03%
sg all: 0.02%
pl n: 0.02%
sg abl: 0.02%
sg e

## Mitu % sõnadest saavad ühese analüüsi

In [70]:
print("EDT korpuse ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(edt_korpusepohistega)
print("\nAjakirjandustekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(aja)
print("\nIlukirjandustekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(ilu)
print("\nTeadustekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(tea)
print("\nEWT korpuse ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(ewt_korpusepohistega)
print("\nMääramatute netitekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(ewtb)
print("\nFoorumite ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(foorum)
print("\nKommentaariumite ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(kom)
print("\nMõlema korpuse peale kokku ühesed analüüsid")
print("---------------------------------------------------------")
analyyse(kogu_korpus)

EDT korpuse ühesed analüüsid
---------------------------------------------------------
Tavalise ja pärisnimede oletamisega, aga ühestamiseta
Üheseid analüüse: 275443 ehk 64.31%

Ühestamisega
Üheseid analüüse: 392215 ehk 91.57%

Ühestamisega, aga ilma postanalysis sammuta
Üheseid analüüse: 392215 ehk 91.57%

Eelühestamisega
Üheseid analüüse: 392891 ehk 91.73%

Järelühestamisega
Üheseid analüüse: 394306 ehk 92.06%

Eel- ja järelühestamisega
Üheseid analüüse: 394266 ehk 92.05%

Eel- ja järelühestamisega ja liitsõnade ühestamise heuristikaga
Üheseid analüüse: 395388 ehk 92.31%

Koprusepõhise ühestamisega
Üheseid analüüse: 395279 ehk 92.29%

Ajakirjandustekstide ühesed analüüsid
---------------------------------------------------------
Tavalise ja pärisnimede oletamisega, aga ühestamiseta
Üheseid analüüse: 167948 ehk 62.93%

Ühestamisega
Üheseid analüüse: 243548 ehk 91.26%

Ühestamisega, aga ilma postanalysis sammuta
Üheseid analüüse: 243548 ehk 91.26%

Eelühestamisega
Üheseid analüüse: 244

## Ühesed ja korrektse analüüsiga

In [79]:
print("EDT korpuse ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(edt_korpusepohistega)
print("\nAjakirjandustekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(aja)
print("\nIlukirjandustekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(ilu)
print("\nTeadustekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(tea)
print("\nEWT korpuse ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(ewt_korpusepohistega)
print("\nMääramatute netitekstide ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(ewtb)
print("\nFoorumite ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(foorum)
print("\nKommentaariumite ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(kom)
print("\nMõlema korpuse peale kokku ühesed analüüsid")
print("---------------------------------------------------------")
analyys_korrektne(kogu_korpus)

EDT korpuse ühesed analüüsid
---------------------------------------------------------
Tavalise ja pärisnimede oletamisega, aga ühestamiseta
Kõik korrektne: 259963 ehk 60.69%
Lemma korrektne: 274353 ehk 64.05%
Sõnaliik korrektne: 273144 ehk 63.77%
Vorm korrektne: 261447 ehk 61.04%

Ühestamisega
Kõik korrektne: 358137 ehk 83.61%
Lemma korrektne: 383566 ehk 89.55%
Sõnaliik korrektne: 380090 ehk 88.74%
Vorm korrektne: 366421 ehk 85.55%

Ühestamisega, aga ilma postanalysis sammuta
Kõik korrektne: 358141 ehk 83.62%
Lemma korrektne: 383586 ehk 89.56%
Sõnaliik korrektne: 380093 ehk 88.74%
Vorm korrektne: 366414 ehk 85.55%

Eelühestamisega
Kõik korrektne: 360834 ehk 84.24%
Lemma korrektne: 386430 ehk 90.22%
Sõnaliik korrektne: 382832 ehk 89.38%
Vorm korrektne: 367434 ehk 85.79%

Järelühestamisega
Kõik korrektne: 359833 ehk 84.01%
Lemma korrektne: 385344 ehk 89.97%
Sõnaliik korrektne: 382042 ehk 89.2%
Vorm korrektne: 368341 ehk 86.0%

Eel- ja järelühestamisega
Kõik korrektne: 361890 ehk 84.49%


Tavalise ja pärisnimede oletamisega, aga ühestamiseta
Kõik korrektne: 18583 ehk 56.83%
Lemma korrektne: 20395 ehk 62.37%
Sõnaliik korrektne: 19935 ehk 60.97%
Vorm korrektne: 19182 ehk 58.66%

Ühestamisega
Kõik korrektne: 25622 ehk 78.36%
Lemma korrektne: 28712 ehk 87.81%
Sõnaliik korrektne: 27705 ehk 84.73%
Vorm korrektne: 27063 ehk 82.77%

Ühestamisega, aga ilma postanalysis sammuta
Kõik korrektne: 25618 ehk 78.35%
Lemma korrektne: 28716 ehk 87.82%
Sõnaliik korrektne: 27701 ehk 84.72%
Vorm korrektne: 27057 ehk 82.75%

Eelühestamisega
Kõik korrektne: 25633 ehk 78.39%
Lemma korrektne: 28719 ehk 87.83%
Sõnaliik korrektne: 27717 ehk 84.77%
Vorm korrektne: 27054 ehk 82.74%

Järelühestamisega
Kõik korrektne: 25656 ehk 78.46%
Lemma korrektne: 28748 ehk 87.92%
Sõnaliik korrektne: 27744 ehk 84.85%
Vorm korrektne: 27110 ehk 82.91%

Eel- ja järelühestamisega
Kõik korrektne: 25664 ehk 78.49%
Lemma korrektne: 28752 ehk 87.93%
Sõnaliik korrektne: 27754 ehk 84.88%
Vorm korrektne: 27101 ehk 82.88%

E

# Lemmad

### Lemmad kirjavahemärkidega, kus liitsõnadel on alakriips alles

In [76]:
print("EDT korpuse lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(loaded_texts_edt_am)
print("\nAjakirjandustekstide lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(aja)
print("\nIlukirjandustekstide lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(ilu)
print("\nTeadustekstide lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(tea)
print("\nEWT korpuse lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(loaded_texts_ewt_am)
print("\nMääramatute netitekstide lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(ewtb)
print("\nFoorumite lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(foorum)
print("\nKommentaariumite lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(kom)
print("\nMõlema korpuse lemmade ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemmad(kogu_korpus)

EDT korpuse lemmade ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5839400049621193 ehk 58.39%
Saagis: 0.9945858110427042 ehk 99.46%
Täpsus: 0.5858021363901387 ehk 58.58%
F1-skoor: 0.7373259127653442 ehk 73.73%

Ühestamisega
Korrektsus: 0.8468735958808457 ehk 84.69%
Saagis: 0.9769003943322617 ehk 97.69%
Täpsus: 0.8641787900567548 ehk 86.42%
F1-skoor: 0.9170888552087819 ehk 91.71%

Eel- ja järelühestamisega
Korrektsus: 0.8592621040310341 ehk 85.93
Saagis: 0.9820530959401754 ehk 98.21
Täpsus: 0.8729702805910676 ehk 87.3
F1-skoor: 0.9243044347196478 ehk 92.43

Ajakirjandustekstide lemmade ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5737145645616467 ehk 57.37%
Saagis: 0.9956569638467534 ehk 99.57%
Täpsus: 0.575153905431498 ehk 57.52%
F1-skoor: 0.7291215033285953 ehk 72.91%

Ühestamisega
Korrektsus: 0.8455254230691

## Sõnaliigid kirjavahemärkidega

In [77]:
print("EDT korpuse sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(loaded_texts_edt_am)
print("\nAjakirjandustekstide sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(aja)
print("\nIlukirjandustekstide sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(ilu)
print("\nTeadustekstide sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(tea)
print("\nEWT korpuse sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(loaded_texts_ewt_am)
print("\nMääramatute netitekstide sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(ewtb)
print("\nFoorumite sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(foorum)
print("\nKommentaariumite sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(kom)
print("\nMõlema korpuse sõnaliikide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
sonaliik(kogu_korpus)

EDT korpuse sõnaliikide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5815920942773847 ehk 58.16%
Saagis: 0.9920573217625182 ehk 99.21%
Täpsus: 0.5843128788462173 ehk 58.43%
F1-skoor: 0.7354514433673986 ehk 73.55%

Ühestamisega
Korrektsus: 0.8355057508719843 ehk 83.55%
Saagis: 0.9697561863937859 ehk 96.98%
Täpsus: 0.8578589308285212 ehk 85.79%
F1-skoor: 0.9103820573431217 ehk 91.04%

Eel- ja järelühestamisega
Korrektsus: 0.8481961913503283 ehk 84.82
Saagis: 0.9752100653951844 ehk 97.52
Täpsus: 0.8668873484974265 ehk 86.69
F1-skoor: 0.9178638018192427 ehk 91.79

Ajakirjandustekstide sõnaliikide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5716885760618293 ehk 57.17%
Saagis: 0.9934198692967204 ehk 99.34%
Täpsus: 0.5738616193189198 ehk 57.39%
F1-skoor: 0.72748327470103 ehk 72.75%

Ühestamisega
Korrektsus: 0.83558

## Vormid kirjavahemärkidega

In [78]:
print("EDT korpuse vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(loaded_texts_edt_am)
print("\nAjakirjandustekstide vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(aja)
print("\nIlukirjandustekstide vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(ilu)
print("\nTeadustekstide vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(tea)
print("\nEWT korpuse vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(loaded_texts_ewt_am)
print("\nMääramatute netitekstide vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(ewtb)
print("\nFoorumite ühesed vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(foorum)
print("\nKommentaariumite vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(kom)
print("\nMõlema korpuse vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
vorm(kogu_korpus)

EDT korpuse vormide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5533719684412737 ehk 55.34%
Saagis: 0.9610687361522603 ehk 96.11%
Täpsus: 0.5660608794182682 ehk 56.61%
F1-skoor: 0.7124783756675525 ehk 71.25%

Ühestamisega
Korrektsus: 0.7839021901263482 ehk 78.39%
Saagis: 0.9361807437914265 ehk 93.62%
Täpsus: 0.8281576577692962 ehk 82.82%
F1-skoor: 0.8788622991385271 ehk 87.89%

Eel- ja järelühestamisega
Korrektsus: 0.7885989855895549 ehk 78.86
Saagis: 0.9368998340022273 ehk 93.69
Täpsus: 0.832832475510543 ehk 83.28
F1-skoor: 0.881806365700938 ehk 88.18

Ajakirjandustekstide vormide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.547655803741116 ehk 54.77%
Saagis: 0.9664360872954014 ehk 96.64%
Täpsus: 0.5582740945338679 ehk 55.83%
F1-skoor: 0.7077229994127623 ehk 70.77%

Ühestamisega
Korrektsus: 0.78876608639344

## Lemma, sõnaliik ja vorm peavad õiged olema kirjavahemärkideta

In [82]:
print("EDT korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(loaded_texts_edt_am)
print("\nAjakirjandustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(aja)
print("\nIlukirjandustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(ilu)
print("\nTeadustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(tea)
print("\nEWT korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(loaded_texts_ewt_am)
print("\nMääramatute netitekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(ewtb)
print("\nFoorumite ühesed lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(foorum)
print("\nKommentaariumite lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(kom)
print("\nMõlema korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(kogu_korpus)

EDT korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5006511553567887 ehk 50.07%
Saagis: 0.946569221342093 ehk 94.66%
Täpsus: 0.5152110942911186 ehk 51.52%
F1-skoor: 0.6672452202760687 ehk 66.72%

Ühestamisega
Korrektsus: 0.7155138282775709 ehk 71.55%
Saagis: 0.8994030727077014 ehk 89.94%
Täpsus: 0.7777570924986581 ehk 77.78%
F1-skoor: 0.8341685347951627 ehk 83.42%

Eel- ja järelühestamisega
Korrektsus: 0.7271971715193201 ehk 72.72
Saagis: 0.905140550122181 ehk 90.51
Täpsus: 0.7871895501120951 ehk 78.72
F1-skoor: 0.8420546113790179 ehk 84.21

Ajakirjandustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.49714452999753717 ehk 49.71%
Saagis: 0.953087695542173 ehk 95.31%
Täpsus: 0.509614880796437 ehk 50.96%
F1-skoor: 0.6641236300657694 ehk 6

## Lemma, sõnaliik ja vorm peavad õiged olema kirjavahemärkidega

In [83]:
print("EDT korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(loaded_texts_edt_am, kirjavahemargid = True)
print("\nAjakirjandustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(aja, kirjavahemargid = True)
print("\nIlukirjandustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(ilu, kirjavahemargid = True)
print("\nTeadustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(tea, kirjavahemargid = True)
print("\nEWT korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(loaded_texts_ewt_am, kirjavahemargid = True)
print("\nMääramatute netitekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(ewtb, kirjavahemargid = True)
print("\nFoorumite ühesed lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(foorum, kirjavahemargid = True)
print("\nKommentaariumite lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(kom, kirjavahemargid = True)
print("\nMõlema korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga")
print("---------------------------------------------------------")
lemma_liik_vorm(kogu_korpus, kirjavahemargid = True)

EDT korpuse lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5464930412384935 ehk 54.65%
Saagis: 0.9538201640734709 ehk 95.38%
Täpsus: 0.5613455847570433 ehk 56.13%
F1-skoor: 0.7067513744527943 ehk 70.68%

Ühestamisega
Korrektsus: 0.7518662304449001 ehk 75.19%
Saagis: 0.9144232183763109 ehk 91.44%
Täpsus: 0.808774690822573 ehk 80.88%
F1-skoor: 0.8583603215571519 ehk 85.84%

Eel- ja järelühestamisega
Korrektsus: 0.7622474033568535 ehk 76.22
Saagis: 0.9192150342792027 ehk 91.92
Täpsus: 0.8169765897393325 ehk 81.7
F1-skoor: 0.8650855741422807 ehk 86.51

Ajakirjandustekstide lemmade, sõnaliikide ja vormide ühildumine kuldstandardiga
---------------------------------------------------------
Oletamistega, aga ilma ühestamiseta
Korrektsus: 0.5414127010656506 ehk 54.14%
Saagis: 0.9598771220452653 ehk 95.99%
Täpsus: 0.5539491570917718 ehk 55.39%
F1-skoor: 0.7024889579427324 ehk 

## Lemmatiseerimise korrektsused sõnaliikide lõikes alakriipsuta

In [36]:
print("EDT korpuse lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(loaded_texts_edt_am)
print("\nAjakirjandustekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(aja)
print("\nIlukirjandustekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(ilu)
print("\nTeadustekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(tea)
print("\nEWT korpuse lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(loaded_texts_ewt_am)
print("\nMääramatute netitekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(ewtb)
print("\nFoorumite lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(foorum)
print("\nKommentaariumite lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(kom)
print("\nMõlema korpuse peale kokku lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(kogu_korpus)

EDT korpuse lemmatiseerimise korrektsus
---------------------------------------------------------
A: 33.13%
C: 97.22%
U: 97.24%
D: 97.6%
G: 51.13%
H: 51.56%
I: 80.34%
J: 99.71%
K: 91.68%
N: 97.62%
O: 79.99%
P: 78.54%
S: 82.2%
V: 72.36%
Y: 86.6%
Z: 99.97%
Käändsõnad: 64.75%
Tegusõnad: 72.36%

Ajakirjandustekstide lemmatiseerimise korrektsus
---------------------------------------------------------
A: 33.02%
C: 98.32%
U: 97.63%
D: 97.64%
G: 47.67%
H: 53.71%
I: 79.72%
J: 99.6%
K: 90.57%
N: 97.86%
O: 82.27%
P: 77.63%
S: 82.18%
V: 72.09%
Y: 82.65%
Z: 99.99%
Käändsõnad: 64.9%
Tegusõnad: 72.09%

Ilukirjandustekstide lemmatiseerimise korrektsus
---------------------------------------------------------
A: 24.51%
C: 95.82%
U: 100.0%
D: 97.42%
G: 77.78%
H: 49.39%
I: 90.8%
J: 99.83%
K: 92.95%
N: 88.8%
O: 89.41%
P: 84.78%
S: 78.72%
V: 85.44%
Y: 31.61%
Z: 100.0%
Käändsõnad: 60.17%
Tegusõnad: 85.44%

Teadustekstide lemmatiseerimise korrektsus
---------------------------------------------------------


## Lemmatiseerimise korrektsused sõnaliikide lõikes alakriipsuga

In [37]:
print("EDT korpuse lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(loaded_texts_edt_am, alakriips = True)
print("\nAjakirjandustekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(aja, alakriips = True)
print("\nIlukirjandustekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(ilu, alakriips = True)
print("\nTeadustekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(tea, alakriips = True)
print("\nEWT korpuse lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(loaded_texts_ewt_am, alakriips = True)
print("\nMääramatute netitekstide lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(ewtb, alakriips = True)
print("\nFoorumite lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(foorum, alakriips = True)
print("\nKommentaariumite lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(kom, alakriips = True)
print("\nMõlema korpuse peale kokku lemmatiseerimise korrektsus")
print("---------------------------------------------------------")
lemmatiseerimine(kogu_korpus, alakriips = True)

EDT korpuse lemmatiseerimise korrektsus
---------------------------------------------------------
A: 32.98%
C: 96.93%
U: 97.24%
D: 97.55%
G: 51.13%
H: 49.6%
I: 80.34%
J: 99.71%
K: 91.68%
N: 97.58%
O: 79.99%
P: 78.53%
S: 81.45%
V: 72.36%
Y: 86.6%
Z: 99.97%
Käändsõnad: 64.11%
Tegusõnad: 72.36%

Ajakirjandustekstide lemmatiseerimise korrektsus
---------------------------------------------------------
A: 32.97%
C: 98.32%
U: 97.63%
D: 97.62%
G: 47.67%
H: 51.47%
I: 79.72%
J: 99.6%
K: 90.57%
N: 97.79%
O: 82.27%
P: 77.63%
S: 81.83%
V: 72.09%
Y: 82.65%
Z: 99.99%
Käändsõnad: 64.33%
Tegusõnad: 72.09%

Ilukirjandustekstide lemmatiseerimise korrektsus
---------------------------------------------------------
A: 24.5%
C: 95.82%
U: 100.0%
D: 97.33%
G: 77.78%
H: 47.89%
I: 90.8%
J: 99.83%
K: 92.95%
N: 88.8%
O: 89.41%
P: 84.74%
S: 78.43%
V: 85.44%
Y: 31.61%
Z: 100.0%
Käändsõnad: 59.93%
Tegusõnad: 85.44%

Teadustekstide lemmatiseerimise korrektsus
---------------------------------------------------------

## Liitsõnad

In [88]:
print("EDT korpuse liitsõnad")
print("---------------------------------------------------------")
liitsona(loaded_texts_edt_am)
print("\nAjakirjandustekstide liitsõnad")
print("---------------------------------------------------------")
liitsona(aja)
print("\nIlukirjandustekstide liitsõnad")
print("---------------------------------------------------------")
liitsona(ilu)
print("\nTeadustekstide liitsõnad")
print("---------------------------------------------------------")
liitsona(tea)
print("\nEWT korpuse liitsõnad")
print("---------------------------------------------------------")
liitsona(loaded_texts_ewt_am)
print("\nMääramatute netitekstide liitsõnad")
print("---------------------------------------------------------")
liitsona(ewtb)
print("\nFoorumite liitsõnad")
print("---------------------------------------------------------")
liitsona(foorum)
print("\nKommentaariumite liitsõnad")
print("---------------------------------------------------------")
liitsona(kom)
print("\nMõlema korpuse peale kokku liitsõnad")
print("---------------------------------------------------------")
liitsona(kogu_korpus)

EDT korpuse liitsõnad
---------------------------------------------------------
Täpselt sama lemma, alakriipsud samas kohas: 93.35%
Sama lemma, aga alakriipsud eri kohtades: 0.12%
Ilma kriipsu(de)ta, aga sama lemma: 1.3%
Erinev lemma, kriipsudega või mitte : 5.23%
Pole liitsõnana analüüsitud, aga sama lemma: 578
Pärisnimesid: 390, nimisõnu: 143

Ajakirjandustekstide liitsõnad
---------------------------------------------------------
Täpselt sama lemma, alakriipsud samas kohas: 93.15%
Sama lemma, aga alakriipsud eri kohtades: 0.05%
Ilma kriipsu(de)ta, aga sama lemma: 1.34%
Erinev lemma, kriipsudega või mitte : 5.46%
Pole liitsõnana analüüsitud, aga sama lemma: 375
Pärisnimesid: 350, nimisõnu: 20

Ilukirjandustekstide liitsõnad
---------------------------------------------------------
Täpselt sama lemma, alakriipsud samas kohas: 94.53%
Sama lemma, aga alakriipsud eri kohtades: 0.0%
Ilma kriipsu(de)ta, aga sama lemma: 0.55%
Erinev lemma, kriipsudega või mitte : 4.92%
Pole liitsõnana analü

## Analüüsi ja ühestamisvead

In [25]:
print("EDT korpuse vead")
print("---------------------------------------------------------")
vigu(loaded_texts_edt_am)
print("\nAjakirjandustekstide vead")
print("---------------------------------------------------------")
vigu(aja)
print("\nIlukirjandustekstide vead")
print("---------------------------------------------------------")
vigu(ilu)
print("\nTeadustekstide vead")
print("---------------------------------------------------------")
vigu(tea)
print("\nEWT korpuse vead")
print("---------------------------------------------------------")
vigu(loaded_texts_ewt_am)
print("\nMääramatute netitekstide vead")
print("---------------------------------------------------------")
vigu(ewtb)
print("\nFoorumite vead")
print("---------------------------------------------------------")
vigu(foorum)
print("\nKommentaariumite vead")
print("---------------------------------------------------------")
vigu(kom)
print("\nMõlema korpuse peale kokku vead")
print("---------------------------------------------------------")
vigu(kogu_korpus)

EDT korpuse vead
---------------------------------------------------------
Kõik kategooriad olid valed: 3333
Analüüsivigu: 409 ehk 12.27%
Ühestamisvigu: 2924 ehk 87.73%

Lemma vale: 9894
Analüüsivigu: 2319 ehk 23.44%
Ühestamisvigu: 7575 ehk 76.56%

Sõnaliik vale: 12963
Analüüsivigu: 3402 ehk 26.24%
Ühestamisvigu: 9561 ehk 73.76%

Vorm vale: 27342
Analüüsivigu: 16675 ehk 60.99%
Ühestamisvigu: 10667 ehk 39.01%

Lemma, liik või vorm vale: 50199
Analüüsivigu: 22396 ehk 44.61%
Ühestamisvigu: 27803 ehk 55.39%

Ajakirjandustekstide vead
---------------------------------------------------------
Kõik kategooriad olid valed: 2085
Analüüsivigu: 219 ehk 10.5%
Ühestamisvigu: 1866 ehk 89.5%

Lemma vale: 6153
Analüüsivigu: 1159 ehk 18.84%
Ühestamisvigu: 4994 ehk 81.16%

Sõnaliik vale: 7824
Analüüsivigu: 1756 ehk 22.44%
Ühestamisvigu: 6068 ehk 77.56%

Vorm vale: 15937
Analüüsivigu: 8957 ehk 56.2%
Ühestamisvigu: 6980 ehk 43.8%

Lemma, liik või vorm vale: 29914
Analüüsivigu: 11872 ehk 39.69%
Ühestamisvi

## Sõnaliikide jaotus

In [37]:
print("EDT korpuse sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(loaded_texts_edt_am)
print("\nAjakirjandustekstide sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(aja)
print("\nIlukirjandustekstide sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(ilu)
print("\nTeadustekstide sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(tea)
print("\nEWT korpuse sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(loaded_texts_ewt_am)
print("\nMääramatute netitekstide sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(ewtb)
print("\nFoorumite sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(foorum)
print("\nKommentaariumite sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(kom)
print("\nMõlema korpuse peale kokku sõnaliikide jaotus")
print("---------------------------------------------------------")
sonaliikide_jaotus(kogu_korpus)

EDT korpuse sõnaliikide jaotus
---------------------------------------------------------
A: 29627 ehk 6.92%
C: 2595 ehk 0.61%
U: 404 ehk 0.09%
D: 40979 ehk 9.57%
G: 640 ehk 0.15%
H: 24199 ehk 5.65%
I: 321 ehk 0.07%
J: 24664 ehk 5.76%
K: 8696 ehk 2.03%
N: 9170 ehk 2.14%
O: 2609 ehk 0.61%
P: 29742 ehk 6.94%
S: 111123 ehk 25.94%
V: 69019 ehk 16.11%
Y: 3949 ehk 0.92%
Z: 70582 ehk 16.48%
Käändsõnad: 210109 ehk 49.05%

Ajakirjandustekstide sõnaliikide jaotus
---------------------------------------------------------
A: 17378 ehk 6.51%
C: 1488 ehk 0.56%
U: 317 ehk 0.12%
D: 25688 ehk 9.63%
G: 510 ehk 0.19%
H: 19187 ehk 7.19%
I: 110 ehk 0.04%
J: 15033 ehk 5.63%
K: 5316 ehk 1.99%
N: 5437 ehk 2.04%
O: 1626 ehk 0.61%
P: 18531 ehk 6.94%
S: 68496 ehk 25.67%
V: 43513 ehk 16.31%
Y: 2132 ehk 0.8%
Z: 42102 ehk 15.78%
Käändsõnad: 132970 ehk 49.83%

Ilukirjandustekstide sõnaliikide jaotus
---------------------------------------------------------
A: 3447 ehk 5.09%
C: 275 ehk 0.41%
U: 30 ehk 0.04%
D: 8328 eh